In [31]:
from C_split_functions import *
from C_econ import *


def objective_function(x, equipment):
    ntur = 85  # 2019 Nabil 93
    ncomp = 82  #  89
    fg_tin = 539.76  # °C 630
    fg_m = 68.75  # kg/s 935
    cooler_pdrop = 1e5  # 0.5e5
    heater_pdrop = 0  # 1e5
    hx_pdrop = 0.5e5  # 1e5
    PENALTY_VALUE = float(1e6)
    splitter = False
    splitter2 = False
    looping = False
    enumerated_equipment = list(enumerate(equipment))
    equipment_length = len(equipment)
    equipment_array = np.array(equipment)
    enthalpies = np.zeros(equipment_length)
    entropies = np.zeros(equipment_length)
    exergies = np.zeros(equipment_length)
    w_comp = np.zeros(equipment_length)
    cost_comp = np.zeros(equipment_length)
    w_tur = np.zeros(equipment_length)
    cost_tur = np.zeros(equipment_length)
    q_cooler = np.zeros(equipment_length)
    cost_cooler = np.zeros(equipment_length)
    dissipation = np.zeros(equipment_length)
    q_heater = np.zeros(equipment_length)
    cost_heater = np.zeros(equipment_length)
    q_hx = np.zeros(equipment_length)
    cost_hx = np.zeros(equipment_length)
    cost_hx2 = np.zeros(equipment_length)
    if 9 in equipment:
        splitter = True
        if 6 in equipment:
            looping = True
    if 10 in equipment:
        splitter2 = True
        looping = True

    def inlet_equipment_check(
        equipment_array, assumed_inlet, Temperatures, enthalpies, entropies
    ):
        if equipment[assumed_inlet] == 7:
            splitter_position = np.where(equipment_array == 9)[0]
            mixer1, mixer2 = np.where(equipment_array == 7)[0]
            if mixer1 == assumed_inlet:
                Temperatures[splitter_position] = Temperatures[mixer1]
                enthalpies[splitter_position] = enthalpies[mixer1]
                entropies[splitter_position] = entropies[mixer1]

        if equipment[assumed_inlet] == 8:
            splitter_position = np.where(equipment_array == 10)[0]
            mixer1, mixer2 = np.where(equipment_array == 8)[0]
            if mixer1 == assumed_inlet:
                Temperatures[splitter_position] = Temperatures[mixer1]
                enthalpies[splitter_position] = enthalpies[mixer1]
                entropies[splitter_position] = entropies[mixer1]

        if equipment[assumed_inlet] == 9:
            splitter_position = np.where(equipment_array == 9)[0]
            Temperatures[splitter_position - 1] = Temperatures[assumed_inlet]
            enthalpies[splitter_position - 1] = enthalpies[assumed_inlet]
            entropies[splitter_position - 1] = entropies[assumed_inlet]

        if equipment[assumed_inlet] == 10:
            splitter_position = np.where(equipment_array == 9)[0]
            Temperatures[splitter_position - 1] = Temperatures[assumed_inlet]
            enthalpies[splitter_position - 1] = enthalpies[assumed_inlet]
            entropies[splitter_position - 1] = entropies[assumed_inlet]
        return Temperatures, enthalpies, entropies

    def c_solver_splitter_mixer_calc(
        splitter_position,
        mixer1,
        mixer2,
        Temperatures,
        Pressures,
        enthalpies,
        entropies,
        mass_flow,
    ):
        if Temperatures[splitter_position - 1] != 0:
            Temperatures[splitter_position] = Temperatures[splitter_position - 1]
            enthalpies[splitter_position] = enthalpies[splitter_position - 1]
            entropies[splitter_position] = entropies[splitter_position - 1]
        # if Temperatures[mixer1] != 0:
        #     Temperatures[splitter_position] = Temperatures[mixer1]
        #     enthalpies[splitter_position] = enthalpies[mixer1]
        #     entropies[splitter_position] = entropies[mixer1]
        Temperatures[mixer1] = Temperatures[splitter_position]
        enthalpies[mixer1] = enthalpies[splitter_position]
        entropies[mixer1] = entropies[splitter_position]
        if (
            Pressures[mixer1 - 1] == Pressures[mixer2 - 1]
            and Temperatures[mixer1 - 1] != 0
            and Temperatures[mixer2 - 1] != 0
        ):
            inlet1 = Fluid(FluidsList.CarbonDioxide).with_state(
                Input.temperature(Temperatures[mixer1 - 1]),
                Input.pressure(Pressures[mixer1 - 1]),
            )
            inlet2 = Fluid(FluidsList.CarbonDioxide).with_state(
                Input.temperature(Temperatures[mixer2 - 1]),
                Input.pressure(Pressures[mixer2 - 1]),
            )
            outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                mass_flow[mixer1 - 1], inlet1, mass_flow[mixer2 - 1], inlet2
            )
            Temperatures[mixer2] = outlet.temperature
            enthalpies[mixer2] = outlet.enthalpy
            entropies[mixer2] = outlet.entropy
        if Temperatures[mixer1 - 1] != 0 and Temperatures[mixer2 - 1] != 0:
            if Pressures[mixer1 - 1] > Pressures[mixer2 - 1]:
                hp_index = mixer1 - 1
                lp_index = mixer2 - 1
            elif Pressures[mixer1 - 1] < Pressures[mixer2 - 1]:
                hp_index = mixer2 - 1
                lp_index = mixer1 - 1
            hp_inlet = (
                Fluid(FluidsList.CarbonDioxide)
                .with_state(
                    Input.temperature(Temperatures[hp_index]),
                    Input.pressure(Pressures[hp_index]),
                )
                .isenthalpic_expansion_to_pressure(Pressures[lp_index])
            )
            lp_inlet = Fluid(FluidsList.CarbonDioxide).with_state(
                Input.temperature(Temperatures[lp_index]),
                Input.pressure(Pressures[lp_index]),
            )
            outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                mass_flow[hp_index], hp_inlet, mass_flow[lp_index], lp_inlet
            )
            Temperatures[mixer2] = outlet.temperature
            enthalpies[mixer2] = outlet.enthalpy
            entropies[mixer2] = outlet.entropy
        return Temperatures, Pressures, enthalpies, entropies

    (
        Pressures,
        Temperatures,
        approach_temp,
        approach_temp_2,
        split_ratio,
        split_ratio_2,
        mass_flow,
        assumed_temperature,
    ) = decision_variable_placement(x, enumerated_equipment, equipment_length, looping)
    Pressures = Pressure_calculation(
        Pressures, equipment, cooler_pdrop, heater_pdrop, hx_pdrop, splitter, splitter2
    )
    if Pressures.prod() == 0:
        # print("Infeasible Pressure")
        return PENALTY_VALUE
    tur_pratio, comp_pratio = tur_comp_pratio(
        enumerated_equipment, Pressures, equipment_length
    )
    if np.any(tur_pratio <= 1) or np.any(comp_pratio <= 1):
        # print("Turbine or Compressor pressure ratio is less than 1")
        return PENALTY_VALUE
    turbine_position = np.where(equipment_array == 1)[0]
    cooler_position = np.where(equipment_array == 2)[0]
    for index in cooler_position:
        enthalpies[index], entropies[index] = enthalpy_entropy(
            Temperatures[index], Pressures[index]
        )
    compressor_position = np.where(equipment_array == 3)[0]
    heater_position = np.where(equipment_array == 4)[0]
    for index in heater_position:
        enthalpies[index], entropies[index] = enthalpy_entropy(
            Temperatures[index], Pressures[index]
        )
    hx_position = np.where(equipment_array == 5)[0]
    hx2_position = np.where(equipment_array == 6)[0]
    # Normal Solver
    while_counter = 0
    while Temperatures.prod() == 0:
        # restructuring this part can be useful, separating splitter information from tur/comp calculation while adding if checks
        # combinnig two power checks within the if check
        (
            Temperatures,
            enthalpies,
            entropies,
            w_tur,
            w_comp,
        ) = turbine_compressor_calculation(
            Temperatures,
            Pressures,
            enthalpies,
            entropies,
            w_tur,
            w_comp,
            equipment,
            ntur,
            ncomp,
            mass_flow,
        )

        if np.any(w_tur < 0) or np.any(w_comp < 0):
            # print("Turbine or Compressor output is less than 0")
            return PENALTY_VALUE

        if splitter == True:
            (Temperatures, enthalpies, entropies) = splitter_mixer_calc(
                Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
            )
        if splitter2 == True:
            (Temperatures, enthalpies, entropies) = splitter_mixer_calc_2(
                Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
            )

        if (
            hx_position.size != 0
            and Temperatures[hx_position[0] - 1] != 0
            and Temperatures[hx_position[1] - 1] != 0
        ):
            hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
            if (
                Temperatures[hotside_index - 1]
                < Temperatures[coldside_index - 1] + approach_temp
            ):
                # print("Infeasible HX1.1")
                return PENALTY_VALUE
            try:
                (
                    Temperatures[hotside_index],
                    enthalpies[hotside_index],
                    entropies[hotside_index],
                    Temperatures[coldside_index],
                    enthalpies[coldside_index],
                    entropies[coldside_index],
                    q_hx[min(hotside_index, coldside_index)],
                ) = HX_calculation(
                    Temperatures[hotside_index - 1],
                    Pressures[hotside_index - 1],
                    enthalpies[hotside_index - 1],
                    Temperatures[coldside_index - 1],
                    Pressures[coldside_index - 1],
                    enthalpies[coldside_index - 1],
                    approach_temp,
                    hx_pdrop,
                    mass_flow[hotside_index],
                    mass_flow[coldside_index],
                )
            except:
                # print("HX1 calculation error")
                return PENALTY_VALUE

        if (
            hx2_position.size != 0
            and Temperatures[hx2_position[0] - 1] != 0
            and Temperatures[hx2_position[1] - 1] != 0
        ):
            hotside_index2, coldside_index2 = hx_side_selection(
                hx2_position, Temperatures
            )
            if (
                Temperatures[hotside_index2 - 1]
                < Temperatures[coldside_index2 - 1] + approach_temp_2
            ):
                # print("Infeasible HX2.1")
                return PENALTY_VALUE
            try:
                (
                    Temperatures[hotside_index2],
                    enthalpies[hotside_index2],
                    entropies[hotside_index2],
                    Temperatures[coldside_index2],
                    enthalpies[coldside_index2],
                    entropies[coldside_index2],
                    q_hx[min(hotside_index2, coldside_index2)],
                ) = HX_calculation(
                    Temperatures[hotside_index2 - 1],
                    Pressures[hotside_index2 - 1],
                    enthalpies[hotside_index2 - 1],
                    Temperatures[coldside_index2 - 1],
                    Pressures[coldside_index2 - 1],
                    enthalpies[coldside_index2 - 1],
                    approach_temp_2,
                    hx_pdrop,
                    mass_flow[hotside_index2],
                    mass_flow[coldside_index2],
                )
            except:
                # print("HX2 calculation error")
                return PENALTY_VALUE
        if while_counter == 3:
            # print("Infeasible Temperatures")
            if looping:
                break
            else:
                return PENALTY_VALUE
        while_counter += 1
    if splitter == True:
        splitter1_position = np.where(equipment_array == 9)[0]
        mixer1, mixer2 = np.where(equipment_array == 7)[0]
    if splitter2 == True:
        splitter2_position = np.where(equipment_array == 10)[0]
        mixer3, mixer4 = np.where(equipment_array == 8)[0]
    # Initial Guess solver
    if Temperatures.prod() == 0 and looping == True:
        print(Temperatures, "Initial Temperatures")
        # print("Entering Looping")
        Temperatures_old = Temperatures.copy()
        Temperatures[hx_position - 1]
        assumed_inlet = (
            hx_position[np.where(Temperatures[hx_position - 1] == 0)[0].item()] - 1
        )
        converged = False
        for while_counter in range(10):
            if converged == True:
                break
            hx_converged = False
            Temperatures = Temperatures_old.copy()
            Temperatures[assumed_inlet] = np.random.uniform(50, 500)
            enthalpies[assumed_inlet], entropies[assumed_inlet] = enthalpy_entropy(
                Temperatures[assumed_inlet], Pressures[assumed_inlet]
            )

            # Branch check
            Temperatures, enthalpies, entropies = inlet_equipment_check(
                equipment_array, assumed_inlet, Temperatures, enthalpies, entropies
            )
            assumed_temperature = Temperatures[assumed_inlet].copy()
            hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
            if (
                Temperatures[hotside_index - 1]
                > Temperatures[coldside_index - 1] + approach_temp
            ):
                try:
                    (
                        Temperatures[hotside_index],
                        enthalpies[hotside_index],
                        entropies[hotside_index],
                        Temperatures[coldside_index],
                        enthalpies[coldside_index],
                        entropies[coldside_index],
                        q_hx[min(hotside_index, coldside_index)],
                    ) = HX_calculation(
                        Temperatures[hotside_index - 1],
                        Pressures[hotside_index - 1],
                        enthalpies[hotside_index - 1],
                        Temperatures[coldside_index - 1],
                        Pressures[coldside_index - 1],
                        enthalpies[coldside_index - 1],
                        approach_temp,
                        hx_pdrop,
                        mass_flow[hotside_index],
                        mass_flow[coldside_index],
                    )
                    hx_converged = True
                    # print(assumed_temperature, "HX1 Converged")
                except:
                    hx_converged = False
                if hx_converged == True:
                    for while_counter2nd in range(10):
                        # print(Temperatures, while_counter2nd)
                        for index in turbine_position:
                            if Temperatures[index - 1] != 0:
                                (
                                    enthalpies[index],
                                    entropies[index],
                                    Temperatures[index],
                                    w_tur[index],
                                ) = turbine(
                                    Temperatures[index - 1],
                                    Pressures[index - 1],
                                    Pressures[index],
                                    ntur,
                                    mass_flow[index],
                                )
                        for index in compressor_position:
                            if Temperatures[index - 1] != 0:
                                (
                                    enthalpies[index],
                                    entropies[index],
                                    Temperatures[index],
                                    w_comp[index],
                                ) = compressor(
                                    Temperatures[index - 1],
                                    Pressures[index - 1],
                                    Pressures[index],
                                    ncomp,
                                    mass_flow[index],
                                )
                        if splitter == True:
                            Temperatures, Pressures, enthalpies, entropies = (
                                c_solver_splitter_mixer_calc(
                                    splitter1_position,
                                    mixer1,
                                    mixer2,
                                    Temperatures,
                                    Pressures,
                                    enthalpies,
                                    entropies,
                                    mass_flow,
                                )
                            )
                        if splitter2 == True:
                            Temperatures, Pressures, enthalpies, entropies = (
                                c_solver_splitter_mixer_calc(
                                    splitter2_position,
                                    mixer3,
                                    mixer4,
                                    Temperatures,
                                    Pressures,
                                    enthalpies,
                                    entropies,
                                    mass_flow,
                                )
                            )
                        if (
                            hx_position.size != 0
                            and Temperatures[hx_position[0] - 1] != 0
                            and Temperatures[hx_position[1] - 1] != 0
                        ):
                            hotside_index, coldside_index = hx_side_selection(
                                hx_position, Temperatures
                            )
                            if (
                                Temperatures[hotside_index - 1]
                                < Temperatures[coldside_index - 1] + approach_temp
                            ):
                                c = PENALTY_VALUE
                            try:
                                (
                                    Temperatures[hotside_index],
                                    enthalpies[hotside_index],
                                    entropies[hotside_index],
                                    Temperatures[coldside_index],
                                    enthalpies[coldside_index],
                                    entropies[coldside_index],
                                    q_hx[min(hotside_index, coldside_index)],
                                ) = HX_calculation(
                                    Temperatures[hotside_index - 1],
                                    Pressures[hotside_index - 1],
                                    enthalpies[hotside_index - 1],
                                    Temperatures[coldside_index - 1],
                                    Pressures[coldside_index - 1],
                                    enthalpies[coldside_index - 1],
                                    approach_temp,
                                    hx_pdrop,
                                    mass_flow[hotside_index],
                                    mass_flow[coldside_index],
                                )
                                # print("Succesful HX1")
                            except:
                                # print("HX1 calculation error")
                                c = PENALTY_VALUE
                        if (
                            hx2_position.size != 0
                            and Temperatures[hx2_position[0] - 1] != 0
                            and Temperatures[hx2_position[1] - 1] != 0
                        ):
                            hotside_index2, coldside_index2 = hx_side_selection(
                                hx2_position, Temperatures
                            )
                            if (
                                Temperatures[hotside_index2 - 1]
                                < Temperatures[coldside_index2 - 1] + approach_temp_2
                            ):
                                c = PENALTY_VALUE
                            try:
                                (
                                    Temperatures[hotside_index2],
                                    enthalpies[hotside_index2],
                                    entropies[hotside_index2],
                                    Temperatures[coldside_index2],
                                    enthalpies[coldside_index2],
                                    entropies[coldside_index2],
                                    q_hx[min(hotside_index2, coldside_index2)],
                                ) = HX_calculation(
                                    Temperatures[hotside_index2 - 1],
                                    Pressures[hotside_index2 - 1],
                                    enthalpies[hotside_index2 - 1],
                                    Temperatures[coldside_index2 - 1],
                                    Pressures[coldside_index2 - 1],
                                    enthalpies[coldside_index2 - 1],
                                    approach_temp_2,
                                    hx_pdrop,
                                    mass_flow[hotside_index2],
                                    mass_flow[coldside_index2],
                                )
                                # print("Succesful HX2")
                            except:
                                # print("HX2 calculation error")
                                c = PENALTY_VALUE

                        if (
                            np.round(Temperatures[assumed_inlet]) < 32
                            or np.round(Temperatures[assumed_inlet]) > 530
                        ):
                            converged = False
                            break
                        if np.round(Temperatures[assumed_inlet]) != np.round(
                            assumed_temperature
                        ):
                            # print("Old assumption", assumed_temperature)
                            # print(Temperatures[assumed_inlet], "Old Temperature")
                            # print(
                            #     Temperatures[assumed_inlet - 1],
                            #     "Previous equipment Temperature",
                            # )
                            Temperatures[assumed_inlet] = (
                                Temperatures[assumed_inlet] * 2 + assumed_temperature
                            ) / 3
                            enthalpies[assumed_inlet], entropies[assumed_inlet] = (
                                enthalpy_entropy(
                                    Temperatures[assumed_inlet],
                                    Pressures[assumed_inlet],
                                )
                            )
                            # Branch check
                            Temperatures, enthalpies, entropies = inlet_equipment_check(
                                equipment_array,
                                assumed_inlet,
                                Temperatures,
                                enthalpies,
                                entropies,
                            )
                            assumed_temperature = Temperatures[assumed_inlet].copy()
                            # print("New assumption", assumed_temperature)
                        else:
                            if Temperatures.prod() != 0 and while_counter2nd > 1:
                                if splitter == True:
                                    splitter_position = np.where(equipment_array == 9)[
                                        0
                                    ]
                                    mixer1, mixer2 = np.where(equipment_array == 7)[0]
                                    if (
                                        Temperatures[mixer1]
                                        == Temperatures[splitter_position]
                                        and Temperatures[splitter_position]
                                        == Temperatures[splitter_position - 1]
                                    ):
                                        if splitter2 == True:
                                            splitter_position = np.where(
                                                equipment_array == 10
                                            )[0]
                                            mixer1, mixer2 = np.where(
                                                equipment_array == 8
                                            )[0]
                                            if (
                                                Temperatures[mixer1]
                                                == Temperatures[splitter_position]
                                                and Temperatures[splitter_position]
                                                == Temperatures[splitter_position - 1]
                                            ):
                                                converged = True
                                                break
                                        else:
                                            converged = True
                                            break
        print(Temperatures, "Final Temperatures")
        print(assumed_temperature, "Final Assumed Temperature")
        print(converged, Temperatures[assumed_inlet])

    if sum(w_tur) < sum(w_comp):
        # print("Negative Net Power Production")
        return PENALTY_VALUE

    for index in cooler_position:
        if Temperatures[index] >= Temperatures[index - 1]:
            # print("Infeasible Cooler")
            return PENALTY_VALUE
    enthalpies, entropies, q_cooler = cooler_calculation(
        cooler_position,
        Temperatures,
        Pressures,
        enthalpies,
        entropies,
        q_cooler,
        cooler_pdrop,
        mass_flow,
    )
    if np.any(q_cooler < 0):
        # print("Negative Cooler Work")
        return PENALTY_VALUE

    for index in heater_position:
        if Temperatures[index] <= Temperatures[index - 1]:
            # print("Infeasible Temperatures for heater")
            return PENALTY_VALUE
    enthalpies, entropies, q_heater = heater_calculation(
        heater_position,
        Temperatures,
        Pressures,
        enthalpies,
        entropies,
        q_heater,
        heater_pdrop,
        mass_flow,
    )

    if np.unique(Temperatures[heater_position]).size != len(
        Temperatures[heater_position]
    ):
        # print("Same Temperature for heater")
        return PENALTY_VALUE

    total_heat = sum(q_heater)
    fg_tout = fg_calculation(fg_m, total_heat)
    if fg_tout < 90:
        # print("Too low stack temperature")
        return PENALTY_VALUE
    cost_tur = turbine_econ(w_tur, Temperatures, cost_tur)
    cost_comp = comp_econ(w_comp, cost_comp)
    cost_cooler = cooler_econ(q_cooler, Temperatures, cost_cooler)
    if np.all(cost_cooler == 0):
        return PENALTY_VALUE
    try:
        cost_heater, fg_mlist, fg_tinlist, fg_toutlist, descending_temp = heater_econ(
            equipment,
            q_heater,
            Temperatures,
            cost_heater,
            heater_position,
            total_heat,
            fg_m,
            fg_tin,
        )
    except:
        # print("Heater calculation error")
        return PENALTY_VALUE
    if hx_position.size != 0:
        cost_hx = hx_econ(q_hx, Temperatures, cost_hx, hotside_index, coldside_index)
        if np.all(cost_hx == 0):
            return PENALTY_VALUE
    if hx2_position.size != 0:
        cost_hx2 = hx_econ(q_hx, Temperatures, cost_hx, hotside_index2, coldside_index2)
        if np.all(cost_hx == 0):
            return PENALTY_VALUE
    pec = cost_tur + cost_hx + cost_cooler + cost_comp + cost_heater + cost_hx2
    prod_capacity = (sum(w_tur) - sum(w_comp)) / 1e6
    zk, cfueltot, lcoe = economics(pec, prod_capacity)
    for i, j, k, l in zip(equipment, Temperatures, mass_flow, pec):
        print(f" {i}, {j:.2f}, {k:.0f}, Cost: {l:.0f}")
    # # Exergy Analysis
    # exergies, e_fgin, e_fgout = exergy_calculation(
    #     mass_flow,
    #     enthalpies,
    #     entropies,
    #     q_heater,
    #     fg_mlist,
    #     fg_tinlist,
    #     fg_toutlist,
    #     equipment_length,
    # )
    # # Thermo-economic Analysis
    # if hx_position == []:
    #     hotside_index = 0
    #     coldside_index = 0
    # m1, m2 = exergoeconomic_calculation(
    #     equipment,
    #     Temperatures,
    #     enumerated_equipment,
    #     exergies,
    #     w_tur,
    #     w_comp,
    #     descending_temp,
    #     e_fgin,
    #     e_fgout,
    #     zk,
    #     hotside_index,
    #     coldside_index,
    # )
    # try:
    #     costs = np.linalg.solve(m1, m2)
    # except:
    #     print("Matrix solution problem")
    #     return PENALTY_VALUE
    # Closs = costs[equipment_length + 1] * min(x for x in e_fgout if x != 0)
    # Cfuel = costs[equipment_length] * FGINLETEXERGY
    Ztot = sum(zk)
    # Cproduct = Cfuel + Ztot - Closs
    Ep = sum(w_tur) - sum(w_comp)
    # print(f"Net Power: {Ep/1e6:.2f} MW")
    # for i, j in enumerated_equipment:
    #     if j == 2:
    #         dissipation[i] = costs[i] * (exergies[i - 1] - exergies[i]) + zk[i]
    # Cdiss = sum(dissipation)
    # lcoe_calculated = (costs[-1] * Ep + Cdiss + Closs) / (Ep / 1e6)
    c = lcoe
    thermal_efficiency = (Ep) / 40.53e6
    if thermal_efficiency < 0.1575:
        j = 100 * (0.30 - thermal_efficiency)
    else:
        j = c + 1 * max(0, 0.1 - sum(q_hx) / sum(q_heater))
    # print("Succesful Completion")
    return c

In [69]:
equipment = [1,5,2,3,5,4]
x = [78.55e5,10.7,32.25,241.30e5,10.7,411.85,93.2]
equipment = [9,6,5,1,7,4,1,5,7,6,2,3] #W2CC
x = [0.542,14.1,14.1,78.55e5,0,483.75,79.33e5,14.1,0,14.1,32.25,240e5,119.2]
# equipment = [9,4,7,6,7,10,4,1,5,8,5,1,8,6,2,3] #Noaman1
# x = [0.3,211.5,0.0,5.2,0.0,0.47,520.4,78e5,7.4,0.0,7.4,78e5,0.0,5.2,32.2,240e5,129.8]
equipment = [9,5,1,7,4,1,5,7,6,2,3,10,4,8,6,8] #ED4
# # x = [0.99, 23.0, 8083650.370567948, 0.0, 394.1753975282282, 8902214.157824906, 23.0, 0.0, 4.0, 32.75299068162392, 30000000.0, 0.01, 329.97434984918607, 0.0, 4.0, 0.0, 160]
x = [0.542,14.1,78.55e5,0,483.75,79.33e5,14.1,0,14.1,32.25,240e5,0.5,400,0,14.1,0,100]
x =[0.5623198665724883, 23.0, 7400000.0, 0.0, 480.9650055930451, 7400000.0, 23.0, 0.0, 22.651074514178212, 32.0, 22674030.21408332, 0.2182940438037221, 293.88010517172813, 0.0, 22.651074514178212, 0.0, 158.8823066511589]
print(objective_function(x, equipment))
# a = results_analysis(x, equipment)

[  0.           0.           0.           0.         480.96500559
 361.86164765   0.           0.           0.          32.
 113.35844752 113.35844752 293.88010517 113.35844752   0.
   0.        ] Initial Temperatures
[295.75467973 325.91520458 217.16954914 295.75467973 480.96500559
 361.86164765 318.75467973 258.17523017 136.00952203  32.
 113.35844752 113.35844752 293.88010517 113.35844752 221.02173396
 248.49856435] Final Temperatures
295.75467973014423 Final Assumed Temperature
False 295.75467973014423
 9, 295.75, 89, Cost: 0
 5, 325.92, 89, Cost: 655245
 1, 217.17, 89, Cost: 2172305
 7, 295.75, 70, Cost: 0
 4, 480.97, 70, Cost: 1675777
 1, 361.86, 70, Cost: 2262934
 5, 318.75, 70, Cost: 0
 7, 258.18, 159, Cost: 0
 6, 136.01, 159, Cost: 2672104
 2, 32.00, 159, Cost: 1309462
 3, 113.36, 159, Cost: 2762833
 10, 113.36, 35, Cost: 0
 4, 293.88, 35, Cost: 743631
 8, 113.36, 124, Cost: 0
 6, 221.02, 124, Cost: 0
 8, 248.50, 159, Cost: 0
189.93422876695053


In [255]:
from C_split_functions import *
from C_econ import *


def objective_function2(x, equipment):
    ntur = 85  # 2019 Nabil 93
    ncomp = 82  #  89
    fg_tin = 539.76  # °C 630
    fg_m = 68.75  # kg/s 935
    cooler_pdrop = 1e5  # 0.5e5
    heater_pdrop = 0  # 1e5
    hx_pdrop = 0.5e5  # 1e5
    PENALTY_VALUE = float(1e6)
    splitter = False
    splitter2 = False
    looping = False
    enumerated_equipment = list(enumerate(equipment))
    equipment_length = len(equipment)
    equipment_array = np.array(equipment)
    enthalpies = np.zeros(equipment_length)
    entropies = np.zeros(equipment_length)
    exergies = np.zeros(equipment_length)
    w_comp = np.zeros(equipment_length)
    cost_comp = np.zeros(equipment_length)
    w_tur = np.zeros(equipment_length)
    cost_tur = np.zeros(equipment_length)
    q_cooler = np.zeros(equipment_length)
    cost_cooler = np.zeros(equipment_length)
    dissipation = np.zeros(equipment_length)
    q_heater = np.zeros(equipment_length)
    cost_heater = np.zeros(equipment_length)
    q_hx = np.zeros(equipment_length)
    cost_hx = np.zeros(equipment_length)
    cost_hx2= np.zeros(equipment_length)
    if 9 in equipment:
        splitter = True
        if 6 in equipment:
            looping = True
    if 10 in equipment:
        splitter2 = True
        looping = True
    
    def inlet_equipment_check(
        equipment_array, assumed_inlet, Temperatures, enthalpies, entropies
    ):
        if equipment[assumed_inlet] == 7:
            splitter_position = np.where(equipment_array == 9)[0]
            mixer1, mixer2 = np.where(equipment_array == 7)[0]
            if mixer1 == assumed_inlet:
                Temperatures[splitter_position] = Temperatures[mixer1]
                enthalpies[splitter_position] = enthalpies[mixer1]
                entropies[splitter_position] = entropies[mixer1]

        if equipment[assumed_inlet] == 8:
            splitter_position = np.where(equipment_array == 10)[0]
            mixer1, mixer2 = np.where(equipment_array == 8)[0]
            if mixer1 == assumed_inlet:
                Temperatures[splitter_position] = Temperatures[mixer1]
                enthalpies[splitter_position] = enthalpies[mixer1]
                entropies[splitter_position] = entropies[mixer1]

        if equipment[assumed_inlet] == 9:
            splitter_position = np.where(equipment_array == 9)[0]
            Temperatures[splitter_position - 1] = Temperatures[assumed_inlet]
            enthalpies[splitter_position - 1] = enthalpies[assumed_inlet]
            entropies[splitter_position - 1] = entropies[assumed_inlet]

        if equipment[assumed_inlet] == 10:
            splitter_position = np.where(equipment_array == 9)[0]
            Temperatures[splitter_position - 1] = Temperatures[assumed_inlet]
            enthalpies[splitter_position - 1] = enthalpies[assumed_inlet]
            entropies[splitter_position - 1] = entropies[assumed_inlet]
        return Temperatures, enthalpies, entropies

    def c_solver_splitter_mixer_calc(splitter_position,mixer1,mixer2,Temperatures,Pressures,enthalpies,entropies,mass_flow):
        if Temperatures[splitter_position - 1] != 0: 
            Temperatures[splitter_position] = Temperatures[splitter_position - 1]
            enthalpies[splitter_position] = enthalpies[splitter_position - 1]
            entropies[splitter_position] = entropies[splitter_position - 1]
        # if Temperatures[mixer1] != 0:
        #     Temperatures[splitter_position] = Temperatures[mixer1]
        #     enthalpies[splitter_position] = enthalpies[mixer1]
        #     entropies[splitter_position] = entropies[mixer1]
        Temperatures[mixer1] = Temperatures[splitter_position]
        enthalpies[mixer1] = enthalpies[splitter_position]
        entropies[mixer1] = entropies[splitter_position]
        if (
            Pressures[mixer1 - 1] == Pressures[mixer2 - 1]
            and Temperatures[mixer1 - 1] != 0
            and Temperatures[mixer2 - 1] != 0
        ):
            inlet1 = Fluid(FluidsList.CarbonDioxide).with_state(
                Input.temperature(Temperatures[mixer1 - 1]),
                Input.pressure(Pressures[mixer1 - 1]),
            )
            inlet2 = Fluid(FluidsList.CarbonDioxide).with_state(
                Input.temperature(Temperatures[mixer2 - 1]),
                Input.pressure(Pressures[mixer2 - 1]),
            )
            outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                mass_flow[mixer1 - 1], inlet1, mass_flow[mixer2 - 1], inlet2
            )
            Temperatures[mixer2] = outlet.temperature
            enthalpies[mixer2] = outlet.enthalpy
            entropies[mixer2] = outlet.entropy
        if (
            Temperatures[mixer1 - 1] != 0
            and Temperatures[mixer2 - 1] != 0
        ):
            if Pressures[mixer1 - 1] > Pressures[mixer2 - 1]:
                hp_index = mixer1 - 1
                lp_index = mixer2 - 1
            elif Pressures[mixer1 - 1] < Pressures[mixer2 - 1]:
                hp_index = mixer2 - 1
                lp_index = mixer1 - 1
            hp_inlet = (
                Fluid(FluidsList.CarbonDioxide)
                .with_state(
                    Input.temperature(Temperatures[hp_index]),
                    Input.pressure(Pressures[hp_index]),
                )
                .isenthalpic_expansion_to_pressure(Pressures[lp_index])
            )
            lp_inlet = Fluid(FluidsList.CarbonDioxide).with_state(
                Input.temperature(Temperatures[lp_index]),
                Input.pressure(Pressures[lp_index]),
            )
            outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                mass_flow[hp_index], hp_inlet, mass_flow[lp_index], lp_inlet
            )
            Temperatures[mixer2] = outlet.temperature
            enthalpies[mixer2] = outlet.enthalpy
            entropies[mixer2] = outlet.entropy
        return Temperatures, Pressures,enthalpies, entropies 
    
    (
        Pressures,
        Temperatures,
        approach_temp,
        approach_temp_2,
        split_ratio,
        split_ratio_2,
        mass_flow,
        assumed_temperature,
    ) = decision_variable_placement(x, enumerated_equipment, equipment_length,looping)
    if looping == True:
        assumed_temperature = x[-2]
    print(assumed_temperature,"initial assumption")
    Pressures = Pressure_calculation(
            Pressures, equipment, cooler_pdrop, heater_pdrop, hx_pdrop, splitter,splitter2
        )
    tur_pratio, comp_pratio = tur_comp_pratio(
        enumerated_equipment, Pressures, equipment_length
    )
    turbine_position = np.where(equipment_array == 1)[0]
    cooler_position = np.where(equipment_array==2)[0]
    for index in cooler_position:
        enthalpies[index], entropies[index] = enthalpy_entropy(
            Temperatures[index], Pressures[index]
        )
    compressor_position = np.where(equipment_array == 3)[0]
    heater_position = np.where(equipment_array==4)[0]
    for index in heater_position:
        enthalpies[index], entropies[index] = enthalpy_entropy(
            Temperatures[index], Pressures[index]
        )
    hx_position = np.where(equipment_array==5)[0]
    hx2_position =np.where(equipment_array==6)[0]
    while_counter = 0
    while Temperatures.prod() == 0:
        # restructuring this part can be useful, separating splitter information from tur/comp calculation while adding if checks
        # combinnig two power checks within the if check
        (
            Temperatures,
            enthalpies,
            entropies,
            w_tur,
            w_comp,
        ) = turbine_compressor_calculation(
            Temperatures,
            Pressures,
            enthalpies,
            entropies,
            w_tur,
            w_comp,
            equipment,
            ntur,
            ncomp,
            mass_flow,
        )

        if splitter == True:
            (Temperatures, enthalpies, entropies) = splitter_mixer_calc(
                Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
            )
        if splitter2 == True:
            (Temperatures, enthalpies, entropies) = splitter_mixer_calc_2(
                Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
            )


        if (
            hx_position.size != 0
            and Temperatures[hx_position[0] - 1] != 0
            and Temperatures[hx_position[1] - 1] != 0
        ):
            hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
            if (
                Temperatures[hotside_index - 1]
                < Temperatures[coldside_index - 1] + approach_temp
            ):
                print("Infeasible HX1.1")
                return PENALTY_VALUE
            try:
                (
                    Temperatures[hotside_index],
                    enthalpies[hotside_index],
                    entropies[hotside_index],
                    Temperatures[coldside_index],
                    enthalpies[coldside_index],
                    entropies[coldside_index],
                    q_hx[min(hotside_index, coldside_index)],
                ) = HX_calculation(
                    Temperatures[hotside_index - 1],
                    Pressures[hotside_index - 1],
                    enthalpies[hotside_index - 1],
                    Temperatures[coldside_index - 1],
                    Pressures[coldside_index - 1],
                    enthalpies[coldside_index - 1],
                    approach_temp,
                    hx_pdrop,
                    mass_flow[hotside_index],
                    mass_flow[coldside_index],
                )
            except:
                print("HX1 calculation error")
                return PENALTY_VALUE

        if (
            hx2_position.size != 0
            and Temperatures[hx2_position[0] - 1] != 0
            and Temperatures[hx2_position[1] - 1] != 0
        ):
            hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
            if (
                Temperatures[hotside_index2 - 1]
                < Temperatures[coldside_index2 - 1] + approach_temp_2
            ):
                print("Infeasible HX2.1")
                return PENALTY_VALUE
            try:
                (
                    Temperatures[hotside_index2],
                    enthalpies[hotside_index2],
                    entropies[hotside_index2],
                    Temperatures[coldside_index2],
                    enthalpies[coldside_index2],
                    entropies[coldside_index2],
                    q_hx[min(hotside_index2, coldside_index2)],
                ) = HX_calculation(
                    Temperatures[hotside_index2 - 1],
                    Pressures[hotside_index2 - 1],
                    enthalpies[hotside_index2 - 1],
                    Temperatures[coldside_index2 - 1],
                    Pressures[coldside_index2 - 1],
                    enthalpies[coldside_index2 - 1],
                    approach_temp_2,
                    hx_pdrop,
                    mass_flow[hotside_index2],
                    mass_flow[coldside_index2],
                )
            except:
                print("HX2 calculation error")
                return PENALTY_VALUE
        if while_counter == 3:
            # print("Infeasible Temperatures")
            break
        while_counter += 1
    if splitter == True:
        splitter1_position = np.where(equipment_array == 9)[0]
        mixer1, mixer2 = np.where(equipment_array == 7)[0]
    if splitter2 == True:
        splitter2_position = np.where(equipment_array == 10)[0]
        mixer3, mixer4 = np.where(equipment_array == 8)[0]
    print(Temperatures,"Initial Temperatures")
    if Temperatures.prod() == 0 and looping == True and Temperatures[hx_position-1].sum() > 0:
        print("Entering Looping")
        assumed_inlet = hx_position[np.where(Temperatures[hx_position-1]==0)[0].item()]-1
        Temperatures[assumed_inlet] = assumed_temperature
        enthalpies[assumed_inlet], entropies[assumed_inlet] = enthalpy_entropy(Temperatures[assumed_inlet], Pressures[assumed_inlet])
        #Branch check
        Temperatures, enthalpies, entropies = inlet_equipment_check(
            equipment_array, assumed_inlet, Temperatures, enthalpies, entropies
        )
        hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
        if (
            Temperatures[hotside_index - 1]
            > Temperatures[coldside_index - 1] + approach_temp
        ):
            try:
                (
                    Temperatures[hotside_index],
                    enthalpies[hotside_index],
                    entropies[hotside_index],
                    Temperatures[coldside_index],
                    enthalpies[coldside_index],
                    entropies[coldside_index],
                    q_hx[min(hotside_index, coldside_index)],
                ) = HX_calculation(
                    Temperatures[hotside_index - 1],
                    Pressures[hotside_index - 1],
                    enthalpies[hotside_index - 1],
                    Temperatures[coldside_index - 1],
                    Pressures[coldside_index - 1],
                    enthalpies[coldside_index - 1],
                    approach_temp,
                    hx_pdrop,
                    mass_flow[hotside_index],
                    mass_flow[coldside_index],
                )
                hx_converged = True
                # print(Temperatures,"HX1 Converged Temperatures")
            except:
                return PENALTY_VALUE
            if hx_converged == True:
                for while_counter2nd in range(5):
                    # print(while_counter2nd,Temperatures)
                    for index in turbine_position:
                        if Temperatures[index-1] != 0:
                            (
                                enthalpies[index],
                                entropies[index],
                                Temperatures[index],
                                w_tur[index],
                            ) = turbine(
                                Temperatures[index-1],
                                Pressures[index-1],
                                Pressures[index],
                                ntur,
                                mass_flow[index],
                            )
                    for index in compressor_position:
                        if Temperatures[index-1] != 0:
                            (
                                enthalpies[index],
                                entropies[index],
                                Temperatures[index],
                                w_comp[index],
                            ) = compressor(
                                Temperatures[index-1],
                                Pressures[index-1],
                                Pressures[index],
                                ncomp,
                                mass_flow[index],
                            )
                    if splitter == True:
                        Temperatures,Pressures,enthalpies,entropies = c_solver_splitter_mixer_calc(splitter1_position,mixer1,mixer2,Temperatures,Pressures,enthalpies,entropies,mass_flow)
                    if splitter2 == True:
                        Temperatures,Pressures,enthalpies,entropies = c_solver_splitter_mixer_calc(splitter2_position,mixer3,mixer4,Temperatures,Pressures,enthalpies,entropies,mass_flow)
                    if (
                        hx_position.size != 0
                        and Temperatures[hx_position[0] - 1] != 0
                        and Temperatures[hx_position[1] - 1] != 0
                    ):
                        hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
                        if (
                            Temperatures[hotside_index - 1]
                            < Temperatures[coldside_index - 1] + approach_temp
                        ):
                            print("Looping Infeasible HX1")
                            return PENALTY_VALUE
                        try:
                            (
                                Temperatures[hotside_index],
                                enthalpies[hotside_index],
                                entropies[hotside_index],
                                Temperatures[coldside_index],
                                enthalpies[coldside_index],
                                entropies[coldside_index],
                                q_hx[min(hotside_index, coldside_index)],
                            ) = HX_calculation(
                                Temperatures[hotside_index - 1],
                                Pressures[hotside_index - 1],
                                enthalpies[hotside_index - 1],
                                Temperatures[coldside_index - 1],
                                Pressures[coldside_index - 1],
                                enthalpies[coldside_index - 1],
                                approach_temp,
                                hx_pdrop,
                                mass_flow[hotside_index],
                                mass_flow[coldside_index],
                            )
                        except:
                            print("Looping HX1 calculation error")
                            return PENALTY_VALUE
                    if (
                        hx2_position.size != 0
                        and Temperatures[hx2_position[0] - 1] != 0
                        and Temperatures[hx2_position[1] - 1] != 0
                    ):
                        hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
                        if (
                            Temperatures[hotside_index2 - 1]
                            < Temperatures[coldside_index2 - 1] + approach_temp_2
                        ):
                            print("Looping Infeasible HX2")
                            return PENALTY_VALUE
                        try:
                            (
                                Temperatures[hotside_index2],
                                enthalpies[hotside_index2],
                                entropies[hotside_index2],
                                Temperatures[coldside_index2],
                                enthalpies[coldside_index2],
                                entropies[coldside_index2],
                                q_hx[min(hotside_index2, coldside_index2)],
                            ) = HX_calculation(
                                Temperatures[hotside_index2 - 1],
                                Pressures[hotside_index2 - 1],
                                enthalpies[hotside_index2 - 1],
                                Temperatures[coldside_index2 - 1],
                                Pressures[coldside_index2 - 1],
                                enthalpies[coldside_index2 - 1],
                                approach_temp_2,
                                hx_pdrop,
                                mass_flow[hotside_index2],
                                mass_flow[coldside_index2],
                            )
                        except:
                            print("Looping HX2 calculation error")
                            return PENALTY_VALUE
        print(Temperatures,"Final Temperatures")
        print(assumed_temperature,"Final assumed temperature")
        print("Converged",Temperatures[assumed_inlet])
        
    for index in cooler_position:
        if Temperatures[index] >= Temperatures[index - 1]:
            print("Infeasible Cooler")
            return PENALTY_VALUE
    enthalpies, entropies, q_cooler = cooler_calculation(
        cooler_position,
        Temperatures,
        Pressures,
        enthalpies,
        entropies,
        q_cooler,
        cooler_pdrop,
        mass_flow,
    )

    for index in heater_position:
        if Temperatures[index] <= Temperatures[index - 1]:
            print("Infeasible Temperatures for heater")
            return PENALTY_VALUE
        else:
            enthalpies, entropies, q_heater = heater_calculation(
                heater_position,
                Temperatures,
                Pressures,
                enthalpies,
                entropies,
                q_heater,
                heater_pdrop,
                mass_flow,
            )

    total_heat = sum(q_heater)
    fg_tout = fg_calculation(fg_m, total_heat)
    cost_tur = turbine_econ(w_tur, Temperatures, cost_tur)
    cost_comp = comp_econ(w_comp, cost_comp)
    cost_cooler = cooler_econ(q_cooler, Temperatures, cost_cooler)
    if np.all(cost_cooler == 0):
        return PENALTY_VALUE
    try:
        cost_heater, fg_mlist, fg_tinlist, fg_toutlist, descending_temp = heater_econ(
            equipment,
            q_heater,
            Temperatures,
            cost_heater,
            heater_position,
            total_heat,
            fg_m,
            fg_tin,
        )
    except:
        # print("Heater calculation error")
        return PENALTY_VALUE
    if hx_position.size != 0:
        cost_hx = hx_econ(q_hx, Temperatures, cost_hx, hotside_index, coldside_index)
        if np.all(cost_hx == 0):
            return PENALTY_VALUE
    if hx2_position.size != 0:
        cost_hx2 = hx_econ(q_hx, Temperatures, cost_hx, hotside_index2, coldside_index2)
        if np.all(cost_hx == 0):
            return PENALTY_VALUE
    pec = cost_tur + cost_hx + cost_cooler + cost_comp + cost_heater+cost_hx2
    prod_capacity = (sum(w_tur) - sum(w_comp)) / 1e6
    zk, cfueltot, lcoe = economics(pec, prod_capacity)
    for i, j, k, l in zip(equipment, Temperatures, mass_flow, pec):
        print(f" {i}, {j:.2f}, {k:.0f}, Cost: {l:.0f}")
    Ep = sum(w_tur) - sum(w_comp)
    c = lcoe
    thermal_efficiency = (Ep) / 40.53e6
    if thermal_efficiency < 0.1575:
        j = 100 * (0.30 - thermal_efficiency)
    else:
        j = c + 1 * max(0, 0.1 - sum(q_hx) / sum(q_heater))
    # print("Succesful Completion")
    return c

In [286]:
equipment = [1,5,2,3,5,4]
x = [78.55e5,10.7,32.25,241.30e5,10.7,411.85,93.2]
equipment = [9,6,5,1,7,4,1,5,7,6,2,3] #W2CC
x = [0.542,14.1,14.1,78.55e5,0,483.75,79.33e5,14.1,0,14.1,32.25,240e5,182.39711967668362,119.2]
# equipment = [9,4,7,6,7,10,4,1,5,8,5,1,8,6,2,3] #Noaman1
# x = [0.3,211.5,0.0,5.2,0.0,0.47,520.4,78e5,7.4,0.0,7.4,78e5,0.0,5.2,32.2,240e5,214.82955469678234,129.8]
equipment = [9,5,1,7,4,1,5,7,6,2,3,10,4,8,6,8] #ED4
# # x = [0.99, 23.0, 8083650.370567948, 0.0, 394.1753975282282, 8902214.157824906, 23.0, 0.0, 4.0, 32.75299068162392, 30000000.0, 0.01, 329.97434984918607, 0.0, 4.0, 0.0, 160]
# x = [0.542,14.1,78.55e5,0,483.75,79.33e5,14.1,0,14.1,32.25,240e5,0.5,400,0,14.1,0,337.987450204182,100]
x = [0.99, 4.0, 7820253.851898901, 0.0, 530.0, 24328973.579785462, 4.0, 0.0, 23.0, 32.0, 30000000.0, 0.01, 180.0, 0.0, 23.0, 0.0, 500, 160]
print(objective_function2(x,equipment))

500 initial assumption
[  0.           0.           0.           0.         530.
 505.03561971   0.           0.           0.          32.
  78.24382879  78.24382879 180.          78.24382879   0.
   0.        ] Initial Temperatures
Entering Looping
[250.61398434 252.84747107 131.63205234 250.61398434 530.
 505.03561971 254.61398434 132.45682904 101.24382879  32.
  78.24382879  78.24382879 180.          78.24382879  97.19132093
  97.93591409] Final Temperatures
500 Final assumed temperature
Converged 250.61398433587067
 9, 250.61, 158, Cost: 0
 5, 252.85, 158, Cost: 157713
 1, 131.63, 158, Cost: 3276652
 7, 250.61, 2, Cost: 0
 4, 530.00, 2, Cost: 35585
 1, 505.04, 2, Cost: 33418
 5, 254.61, 2, Cost: 0
 7, 132.46, 160, Cost: 0
 6, 101.24, 160, Cost: 1045338
 2, 32.00, 160, Cost: 2154764
 3, 78.24, 160, Cost: 2552920
 10, 78.24, 2, Cost: 0
 4, 180.00, 2, Cost: 3842
 8, 78.24, 158, Cost: 0
 6, 97.19, 158, Cost: 0
 8, 97.94, 160, Cost: 0
199.28460401225595


In [29]:
#old OBJ function
from C_split_functions import *
from C_econ import *


def objective_function(x, equipment):
    ntur = 85  # 2019 Nabil 93
    ncomp = 82  #  89
    fg_tin = 539.76  # °C 630
    fg_m = 68.75  # kg/s 935
    cooler_pdrop = 1e5  # 0.5e5
    heater_pdrop = 0  # 1e5
    hx_pdrop = 0.5e5  # 1e5
    PENALTY_VALUE = float(1e6)
    splitter = False
    splitter2 = False
    looping = False
    enumerated_equipment = list(enumerate(equipment))
    equipment_length = len(equipment)
    equipment_array = np.array(equipment)
    enthalpies = np.zeros(equipment_length)
    entropies = np.zeros(equipment_length)
    exergies = np.zeros(equipment_length)
    w_comp = np.zeros(equipment_length)
    cost_comp = np.zeros(equipment_length)
    w_tur = np.zeros(equipment_length)
    cost_tur = np.zeros(equipment_length)
    q_cooler = np.zeros(equipment_length)
    cost_cooler = np.zeros(equipment_length)
    dissipation = np.zeros(equipment_length)
    q_heater = np.zeros(equipment_length)
    cost_heater = np.zeros(equipment_length)
    q_hx = np.zeros(equipment_length)
    cost_hx = np.zeros(equipment_length)
    cost_hx2= np.zeros(equipment_length)
    if 9 in equipment:
        splitter = True
        if 6 in equipment:
            looping = True
    if 10 in equipment:
        splitter2 = True
        looping = True
    
    def inlet_equipment_check(
        equipment_array, assumed_inlet, Temperatures, enthalpies, entropies
    ):
        if equipment[assumed_inlet] == 7:
            splitter_position = np.where(equipment_array == 9)[0]
            mixer1, mixer2 = np.where(equipment_array == 7)[0]
            if mixer1 == assumed_inlet:
                Temperatures[splitter_position] = Temperatures[mixer1]
                enthalpies[splitter_position] = enthalpies[mixer1]
                entropies[splitter_position] = entropies[mixer1]

        if equipment[assumed_inlet] == 8:
            splitter_position = np.where(equipment_array == 10)[0]
            mixer1, mixer2 = np.where(equipment_array == 8)[0]
            if mixer1 == assumed_inlet:
                Temperatures[splitter_position] = Temperatures[mixer1]
                enthalpies[splitter_position] = enthalpies[mixer1]
                entropies[splitter_position] = entropies[mixer1]

        if equipment[assumed_inlet] == 9:
            splitter_position = np.where(equipment_array == 9)[0]
            Temperatures[splitter_position - 1] = Temperatures[assumed_inlet]
            enthalpies[splitter_position - 1] = enthalpies[assumed_inlet]
            entropies[splitter_position - 1] = entropies[assumed_inlet]

        if equipment[assumed_inlet] == 10:
            splitter_position = np.where(equipment_array == 9)[0]
            Temperatures[splitter_position - 1] = Temperatures[assumed_inlet]
            enthalpies[splitter_position - 1] = enthalpies[assumed_inlet]
            entropies[splitter_position - 1] = entropies[assumed_inlet]
        return Temperatures, enthalpies, entropies

    def c_solver_splitter_mixer_calc(splitter_position,mixer1,mixer2,Temperatures,Pressures,enthalpies,entropies,mass_flow):
        if Temperatures[splitter_position - 1] != 0: 
            Temperatures[splitter_position] = Temperatures[splitter_position - 1]
            enthalpies[splitter_position] = enthalpies[splitter_position - 1]
            entropies[splitter_position] = entropies[splitter_position - 1]
        # if Temperatures[mixer1] != 0:
        #     Temperatures[splitter_position] = Temperatures[mixer1]
        #     enthalpies[splitter_position] = enthalpies[mixer1]
        #     entropies[splitter_position] = entropies[mixer1]
        Temperatures[mixer1] = Temperatures[splitter_position]
        enthalpies[mixer1] = enthalpies[splitter_position]
        entropies[mixer1] = entropies[splitter_position]
        if (
            Pressures[mixer1 - 1] == Pressures[mixer2 - 1]
            and Temperatures[mixer1 - 1] != 0
            and Temperatures[mixer2 - 1] != 0
        ):
            inlet1 = Fluid(FluidsList.CarbonDioxide).with_state(
                Input.temperature(Temperatures[mixer1 - 1]),
                Input.pressure(Pressures[mixer1 - 1]),
            )
            inlet2 = Fluid(FluidsList.CarbonDioxide).with_state(
                Input.temperature(Temperatures[mixer2 - 1]),
                Input.pressure(Pressures[mixer2 - 1]),
            )
            outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                mass_flow[mixer1 - 1], inlet1, mass_flow[mixer2 - 1], inlet2
            )
            Temperatures[mixer2] = outlet.temperature
            enthalpies[mixer2] = outlet.enthalpy
            entropies[mixer2] = outlet.entropy
        if (
            Temperatures[mixer1 - 1] != 0
            and Temperatures[mixer2 - 1] != 0
        ):
            if Pressures[mixer1 - 1] > Pressures[mixer2 - 1]:
                hp_index = mixer1 - 1
                lp_index = mixer2 - 1
            elif Pressures[mixer1 - 1] < Pressures[mixer2 - 1]:
                hp_index = mixer2 - 1
                lp_index = mixer1 - 1
            hp_inlet = (
                Fluid(FluidsList.CarbonDioxide)
                .with_state(
                    Input.temperature(Temperatures[hp_index]),
                    Input.pressure(Pressures[hp_index]),
                )
                .isenthalpic_expansion_to_pressure(Pressures[lp_index])
            )
            lp_inlet = Fluid(FluidsList.CarbonDioxide).with_state(
                Input.temperature(Temperatures[lp_index]),
                Input.pressure(Pressures[lp_index]),
            )
            outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                mass_flow[hp_index], hp_inlet, mass_flow[lp_index], lp_inlet
            )
            Temperatures[mixer2] = outlet.temperature
            enthalpies[mixer2] = outlet.enthalpy
            entropies[mixer2] = outlet.entropy
        return Temperatures, Pressures,enthalpies, entropies
    
    (
        Pressures,
        Temperatures,
        approach_temp,
        approach_temp_2,
        split_ratio,
        split_ratio_2,
        mass_flow,
        assumed_temperature,
    ) = decision_variable_placement(x, enumerated_equipment, equipment_length,looping)
    Pressures = Pressure_calculation(
            Pressures, equipment, cooler_pdrop, heater_pdrop, hx_pdrop, splitter,splitter2
        )
    tur_pratio, comp_pratio = tur_comp_pratio(
        enumerated_equipment, Pressures, equipment_length
    )
    turbine_position = np.where(equipment_array == 1)[0]
    cooler_position = np.where(equipment_array==2)[0]
    for index in cooler_position:
        enthalpies[index], entropies[index] = enthalpy_entropy(
            Temperatures[index], Pressures[index]
        )
    compressor_position = np.where(equipment_array == 3)[0]
    heater_position = np.where(equipment_array==4)[0]
    for index in heater_position:
        enthalpies[index], entropies[index] = enthalpy_entropy(
            Temperatures[index], Pressures[index]
        )
    hx_position = np.where(equipment_array==5)[0]
    hx2_position =np.where(equipment_array==6)[0]
    while_counter = 0
    while Temperatures.prod() == 0:
        # restructuring this part can be useful, separating splitter information from tur/comp calculation while adding if checks
        # combinnig two power checks within the if check
        (
            Temperatures,
            enthalpies,
            entropies,
            w_tur,
            w_comp,
        ) = turbine_compressor_calculation(
            Temperatures,
            Pressures,
            enthalpies,
            entropies,
            w_tur,
            w_comp,
            equipment,
            ntur,
            ncomp,
            mass_flow,
        )

        if splitter == True:
            (Temperatures, enthalpies, entropies) = splitter_mixer_calc(
                Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
            )
        if splitter2 == True:
            (Temperatures, enthalpies, entropies) = splitter_mixer_calc_2(
                Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
            )


        if (
            hx_position.size != 0
            and Temperatures[hx_position[0] - 1] != 0
            and Temperatures[hx_position[1] - 1] != 0
        ):
            hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
            if (
                Temperatures[hotside_index - 1]
                < Temperatures[coldside_index - 1] + approach_temp
            ):
                # print("Infeasible HX1.1")
                return PENALTY_VALUE
            try:
                (
                    Temperatures[hotside_index],
                    enthalpies[hotside_index],
                    entropies[hotside_index],
                    Temperatures[coldside_index],
                    enthalpies[coldside_index],
                    entropies[coldside_index],
                    q_hx[min(hotside_index, coldside_index)],
                ) = HX_calculation(
                    Temperatures[hotside_index - 1],
                    Pressures[hotside_index - 1],
                    enthalpies[hotside_index - 1],
                    Temperatures[coldside_index - 1],
                    Pressures[coldside_index - 1],
                    enthalpies[coldside_index - 1],
                    approach_temp,
                    hx_pdrop,
                    mass_flow[hotside_index],
                    mass_flow[coldside_index],
                )
            except:
                # print("HX1 calculation error")
                return PENALTY_VALUE

        if (
            hx2_position.size != 0
            and Temperatures[hx2_position[0] - 1] != 0
            and Temperatures[hx2_position[1] - 1] != 0
        ):
            hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
            if (
                Temperatures[hotside_index2 - 1]
                < Temperatures[coldside_index2 - 1] + approach_temp_2
            ):
                # print("Infeasible HX2.1")
                return PENALTY_VALUE
            try:
                (
                    Temperatures[hotside_index2],
                    enthalpies[hotside_index2],
                    entropies[hotside_index2],
                    Temperatures[coldside_index2],
                    enthalpies[coldside_index2],
                    entropies[coldside_index2],
                    q_hx[min(hotside_index2, coldside_index2)],
                ) = HX_calculation(
                    Temperatures[hotside_index2 - 1],
                    Pressures[hotside_index2 - 1],
                    enthalpies[hotside_index2 - 1],
                    Temperatures[coldside_index2 - 1],
                    Pressures[coldside_index2 - 1],
                    enthalpies[coldside_index2 - 1],
                    approach_temp_2,
                    hx_pdrop,
                    mass_flow[hotside_index2],
                    mass_flow[coldside_index2],
                )
            except:
                print("HX2 calculation error")
                return PENALTY_VALUE
        if while_counter == 3:
            # print("Infeasible Temperatures")
            break
        while_counter += 1
    if splitter == True:
        splitter1_position = np.where(equipment_array == 9)[0]
        mixer1, mixer2 = np.where(equipment_array == 7)[0]
    if splitter2 == True:
        splitter2_position = np.where(equipment_array == 10)[0]
        mixer3, mixer4 = np.where(equipment_array == 8)[0]
    if Temperatures.prod() == 0 and looping == True:
        print(Temperatures,"Initial Temperatures")
        print("Entering Looping")
        Temperatures_old = Temperatures.copy()
        assumed_inlet = hx_position[np.where(Temperatures[hx_position-1]==0)[0].item()]-1
        converged = False
        while converged == False:# for while_counter in range(10):
            hx_converged = False
            Temperatures = Temperatures_old.copy()
            Temperatures[assumed_inlet] = np.random.uniform(50,500)
            enthalpies[assumed_inlet], entropies[assumed_inlet] = enthalpy_entropy(Temperatures[assumed_inlet], Pressures[assumed_inlet])
            #Branch check
            Temperatures, enthalpies, entropies = inlet_equipment_check(
                equipment_array, assumed_inlet, Temperatures, enthalpies, entropies
            )
            assumed_temperature = Temperatures[assumed_inlet].copy()
            hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
            if (
                Temperatures[hotside_index - 1]
                > Temperatures[coldside_index - 1] + approach_temp
            ):
                try:
                    (
                        Temperatures[hotside_index],
                        enthalpies[hotside_index],
                        entropies[hotside_index],
                        Temperatures[coldside_index],
                        enthalpies[coldside_index],
                        entropies[coldside_index],
                        q_hx[min(hotside_index, coldside_index)],
                    ) = HX_calculation(
                        Temperatures[hotside_index - 1],
                        Pressures[hotside_index - 1],
                        enthalpies[hotside_index - 1],
                        Temperatures[coldside_index - 1],
                        Pressures[coldside_index - 1],
                        enthalpies[coldside_index - 1],
                        approach_temp,
                        hx_pdrop,
                        mass_flow[hotside_index],
                        mass_flow[coldside_index],
                    )
                    hx_converged = True
                    # print(assumed_temperature,"HX1 Converged")
                except:
                    hx_converged = False
                if hx_converged == True:
                    for while_counter2nd in range(10):
                        # print(Temperatures,while_counter2nd)
                        for index in turbine_position:
                            if Temperatures[index-1] != 0:
                                (
                                    enthalpies[index],
                                    entropies[index],
                                    Temperatures[index],
                                    w_tur[index],
                                ) = turbine(
                                    Temperatures[index-1],
                                    Pressures[index-1],
                                    Pressures[index],
                                    ntur,
                                    mass_flow[index],
                                )
                        for index in compressor_position:
                            if Temperatures[index-1] != 0:
                                (
                                    enthalpies[index],
                                    entropies[index],
                                    Temperatures[index],
                                    w_comp[index],
                                ) = compressor(
                                    Temperatures[index-1],
                                    Pressures[index-1],
                                    Pressures[index],
                                    ncomp,
                                    mass_flow[index],
                                )
                        if splitter == True:
                            Temperatures,Pressures,enthalpies,entropies = c_solver_splitter_mixer_calc(splitter1_position,mixer1,mixer2,Temperatures,Pressures,enthalpies,entropies,mass_flow)
                        if splitter2 == True:
                            Temperatures,Pressures,enthalpies,entropies = c_solver_splitter_mixer_calc(splitter2_position,mixer3,mixer4,Temperatures,Pressures,enthalpies,entropies,mass_flow)
                        if (
                            hx_position.size != 0
                            and Temperatures[hx_position[0] - 1] != 0
                            and Temperatures[hx_position[1] - 1] != 0
                        ):
                            hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
                            if (
                                Temperatures[hotside_index - 1]
                                < Temperatures[coldside_index - 1] + approach_temp
                            ):
                                c = PENALTY_VALUE
                            try:
                                (
                                    Temperatures[hotside_index],
                                    enthalpies[hotside_index],
                                    entropies[hotside_index],
                                    Temperatures[coldside_index],
                                    enthalpies[coldside_index],
                                    entropies[coldside_index],
                                    q_hx[min(hotside_index, coldside_index)],
                                ) = HX_calculation(
                                    Temperatures[hotside_index - 1],
                                    Pressures[hotside_index - 1],
                                    enthalpies[hotside_index - 1],
                                    Temperatures[coldside_index - 1],
                                    Pressures[coldside_index - 1],
                                    enthalpies[coldside_index - 1],
                                    approach_temp,
                                    hx_pdrop,
                                    mass_flow[hotside_index],
                                    mass_flow[coldside_index],
                                )
                                # print("Succesful HX1")
                            except:
                                # print("HX1 calculation error")
                                c = PENALTY_VALUE
                        if (
                            hx2_position.size != 0
                            and Temperatures[hx2_position[0] - 1] != 0
                            and Temperatures[hx2_position[1] - 1] != 0
                        ):
                            hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
                            if (
                                Temperatures[hotside_index2 - 1]
                                < Temperatures[coldside_index2 - 1] + approach_temp_2
                            ):
                                c = PENALTY_VALUE
                            try:
                                (
                                    Temperatures[hotside_index2],
                                    enthalpies[hotside_index2],
                                    entropies[hotside_index2],
                                    Temperatures[coldside_index2],
                                    enthalpies[coldside_index2],
                                    entropies[coldside_index2],
                                    q_hx[min(hotside_index2, coldside_index2)],
                                ) = HX_calculation(
                                    Temperatures[hotside_index2 - 1],
                                    Pressures[hotside_index2 - 1],
                                    enthalpies[hotside_index2 - 1],
                                    Temperatures[coldside_index2 - 1],
                                    Pressures[coldside_index2 - 1],
                                    enthalpies[coldside_index2 - 1],
                                    approach_temp_2,
                                    hx_pdrop,
                                    mass_flow[hotside_index2],
                                    mass_flow[coldside_index2],
                                )
                                # print("Succesful HX2")
                            except:
                                # print("HX2 calculation error")
                                c = PENALTY_VALUE
                         
                        if np.round(Temperatures[assumed_inlet]) <32 or np.round(Temperatures[assumed_inlet]) > 530:
                                converged = False
                                break
                        if np.round(Temperatures[assumed_inlet]) != np.round(assumed_temperature):
                            # print("Old assumption",assumed_temperature)
                            Temperatures[assumed_inlet] = (Temperatures[assumed_inlet]*2 + assumed_temperature)/3
                            enthalpies[assumed_inlet], entropies[assumed_inlet] = enthalpy_entropy(Temperatures[assumed_inlet], Pressures[assumed_inlet])
                            #Branch check
                            Temperatures, enthalpies, entropies = inlet_equipment_check(equipment_array, assumed_inlet, Temperatures, enthalpies, entropies)
                            assumed_temperature = Temperatures[assumed_inlet].copy()
                            # print("New assumption",assumed_temperature)
                        else:
                            if Temperatures.prod() != 0 and while_counter2nd>1:
                                if splitter == True:
                                    splitter_position = np.where(equipment_array == 9)[0]
                                    mixer1, mixer2 = np.where(equipment_array == 7)[0]
                                    if Temperatures[mixer1] == Temperatures[splitter_position] and Temperatures[splitter_position] == Temperatures[splitter_position - 1]:
                                        if splitter2 == True:
                                            splitter_position = np.where(equipment_array == 10)[0]
                                            mixer1, mixer2 = np.where(equipment_array == 8)[0]
                                            if Temperatures[mixer1] == Temperatures[splitter_position] and Temperatures[splitter_position] == Temperatures[splitter_position - 1]:
                                                converged = True
                                                break
                                        else:
                                            converged = True
                                            break
        print(Temperatures,"Final Temperatures")
        print(assumed_temperature,"Final Assumed Temperature")
        print(converged,Temperatures[assumed_inlet])
    for index in cooler_position:
        if Temperatures[index] >= Temperatures[index - 1]:
            # print("Infeasible Cooler")
            return PENALTY_VALUE
        
    enthalpies, entropies, q_cooler = cooler_calculation(
        cooler_position,
        Temperatures,
        Pressures,
        enthalpies,
        entropies,
        q_cooler,
        cooler_pdrop,
        mass_flow,
    )

    for index in heater_position:
        if Temperatures[index] <= Temperatures[index - 1]:
            # print("Infeasible Temperatures for heater")
            return PENALTY_VALUE
    enthalpies, entropies, q_heater = heater_calculation(
        heater_position,
        Temperatures,
        Pressures,
        enthalpies,
        entropies,
        q_heater,
        heater_pdrop,
        mass_flow,
    )

    total_heat = sum(q_heater)
    fg_tout = fg_calculation(fg_m, total_heat)
    cost_tur = turbine_econ(w_tur, Temperatures, cost_tur)
    cost_comp = comp_econ(w_comp, cost_comp)
    cost_cooler = cooler_econ(q_cooler, Temperatures, cost_cooler)
    if np.all(cost_cooler == 0):
        return PENALTY_VALUE
    try:
        cost_heater, fg_mlist, fg_tinlist, fg_toutlist, descending_temp = heater_econ(
            equipment,
            q_heater,
            Temperatures,
            cost_heater,
            heater_position,
            total_heat,
            fg_m,
            fg_tin,
        )
    except:
        # print("Heater calculation error")
        return PENALTY_VALUE
    if hx_position.size != 0:
        cost_hx = hx_econ(q_hx, Temperatures, cost_hx, hotside_index, coldside_index)
        if np.all(cost_hx == 0):
            return PENALTY_VALUE
    if hx2_position.size != 0:
        cost_hx2 = hx_econ(q_hx, Temperatures, cost_hx, hotside_index2, coldside_index2)
        if np.all(cost_hx == 0):
            return PENALTY_VALUE
    pec = cost_tur + cost_hx + cost_cooler + cost_comp + cost_heater+cost_hx2
    prod_capacity = (sum(w_tur) - sum(w_comp)) / 1e6
    zk, cfueltot, lcoe = economics(pec, prod_capacity)
    for i, j, k, l in zip(equipment, Temperatures, mass_flow, pec):
        print(f" {i}, {j:.2f}, {k:.0f}, Cost: {l:.0f}")
    # # Exergy Analysis
    # exergies, e_fgin, e_fgout = exergy_calculation(
    #     mass_flow,
    #     enthalpies,
    #     entropies,
    #     q_heater,
    #     fg_mlist,
    #     fg_tinlist,
    #     fg_toutlist,
    #     equipment_length,
    # )
    # # Thermo-economic Analysis
    # if hx_position == []:
    #     hotside_index = 0
    #     coldside_index = 0
    # m1, m2 = exergoeconomic_calculation(
    #     equipment,
    #     Temperatures,
    #     enumerated_equipment,
    #     exergies,
    #     w_tur,
    #     w_comp,
    #     descending_temp,
    #     e_fgin,
    #     e_fgout,
    #     zk,
    #     hotside_index,
    #     coldside_index,
    # )
    # try:
    #     costs = np.linalg.solve(m1, m2)
    # except:
    #     print("Matrix solution problem")
    #     return PENALTY_VALUE
    # Closs = costs[equipment_length + 1] * min(x for x in e_fgout if x != 0)
    # Cfuel = costs[equipment_length] * FGINLETEXERGY
    Ztot = sum(zk)
    # Cproduct = Cfuel + Ztot - Closs
    Ep = sum(w_tur) - sum(w_comp)
    # for i, j in enumerated_equipment:
    #     if j == 2:
    #         dissipation[i] = costs[i] * (exergies[i - 1] - exergies[i]) + zk[i]
    # Cdiss = sum(dissipation)
    # lcoe_calculated = (costs[-1] * Ep + Cdiss + Closs) / (Ep / 1e6)
    c = lcoe
    thermal_efficiency = (Ep) / 40.53e6
    if thermal_efficiency < 0.1575:
        j = 100 * (0.30 - thermal_efficiency)
    else:
        j = c + 1 * max(0, 0.1 - sum(q_hx) / sum(q_heater))
    # print("Succesful Completion")
    return c

In [30]:
equipment = [9,5,1,7,4,1,5,7,6,2,3,10,4,8,6,8] #ED4
# # x = [0.99, 23.0, 8083650.370567948, 0.0, 394.1753975282282, 8902214.157824906, 23.0, 0.0, 4.0, 32.75299068162392, 30000000.0, 0.01, 329.97434984918607, 0.0, 4.0, 0.0, 160]
x = [0.542,14.1,78.55e5,0,483.75,79.33e5,14.1,0,14.1,32.25,240e5,0.5,400,0,14.1,0,100]
print(objective_function(x, equipment))

[  0.           0.           0.           0.         483.75
 365.25444713   0.           0.           0.          32.25
  70.23430959  70.23430959 400.          70.23430959   0.
   0.        ] Initial Temperatures
Entering Looping
[337.9859633  348.03623809 239.68962579 337.9859633  483.75
 365.25444713 352.0859633  291.34902449 137.11029164  32.25
  70.23430959  70.23430959 400.          70.23430959 277.24902449
 337.9859633 ] Final Temperatures
337.9223060937209 Final Assumed Temperature
True 337.9859633028391
 9, 337.99, 54, Cost: 0
 5, 348.04, 54, Cost: 175613
 1, 239.69, 54, Cost: 1518761
 7, 337.99, 46, Cost: 0
 4, 483.75, 46, Cost: 559412
 1, 365.25, 46, Cost: 1613618
 5, 352.09, 46, Cost: 0
 7, 291.35, 100, Cost: 0
 6, 137.11, 100, Cost: 2023673
 2, 32.25, 100, Cost: 1208456
 3, 70.23, 100, Cost: 1892895
 10, 70.23, 50, Cost: 0
 4, 400.00, 50, Cost: 4221615
 8, 70.23, 50, Cost: 0
 6, 277.25, 50, Cost: 0
 8, 337.99, 100, Cost: 0
211.86993117288372


In [ ]:
import numpy as np
from C_split_functions import *
from C_econ import *
"""
Cascaded cycle (W2) back-to-back HX could not work with this implementation
equipment = [9,6,5,1,7,4,1,5,7,6,2,3]
x = [0.542,14.1,14.1,78.55e5,0,483.75,79.33e5,14.1,0,14.1,32.25,240e5,119.2]
works with improved one
Dual Recuperated (W3) works
# equipment = [9,5,1,6,7,6,4,1,5,7,2,3]
# x = [0.428,23,78.55e5,23,0,23,470.35,78.55e5,23,0,32.25,240e5,112.77] # Paper numbers
# x = [0.4175135091747311, 23.0, 7933875.007504854, 20.013322858957036, 0.0, 20.013322858957036, 479.7526393928587, 7931130.82668967, 20.013322858957036, 0.0, 32.0, 30000000.0, 96.92876484580881]

Cascade 3 Noaman CS4 did not work
equipment = [9,4,7,6,7,10,4,1,5,8,5,1,8,6,2,3]
x = [0.3,211.5,0.0,5.2,0.0,0.47,520.4,78e5,7.4,0.0,7.4,78e5,0.0,5.2,32.2,240e5,129.8,]

Noaman example string p125
equipment = [9,3,7,2,3,6,7,5,4,1,5,6]
ED4 did not work
equipment = [9,5,1,7,4,1,5,7,6,2,3,10,4,8,6,8]
x = [0.542,14.1,78.55e5,0,483.75,79.33e5,14.1,0,14.1,32.25,240e5,0.5,400,0,14.1,0,100]
Turchi 1 works
equipment = [1,5,2,3,5,4,1,4]
Turchi 2 does not work
equipment = [9,2,3,6,7,3,7,5,4,1,4,1,5,6]
Turchi 3 does not work
equipment = [9,2,3,6,7,3,7,5,4,1,4,1,5,6,2]
Turchi 4
equipment = [9,2,3,2,3,6,7,3,7,5,4,1,4,1,5,6]
"""
equipment = [9,6,5,1,7,4,1,5,7,6,2,3]
x = [0.542,14.1,14.1,78.55e5,0,483.75,79.33e5,14.1,0,14.1,32.25,240e5,119.2]
ntur = 85  # 2019 Nabil 93
ncomp = 82  #  89
fg_tin = 539.76  # °C 630
fg_m = 68.75  # kg/s 935
cooler_pdrop = 1e5  # 0.5e5
heater_pdrop = 0  # 1e5
hx_pdrop = 0.5e5  # 1e5
PENALTY_VALUE = float(1e6)
splitter = False
splitter2 = False
if 9 in equipment:
    splitter = True
if 10 in equipment:
    splitter2 = True
enumerated_equipment = list(enumerate(equipment))
equipment_length = len(equipment)
equipment_array = np.array(equipment)
enthalpies = np.zeros(equipment_length)
entropies = np.zeros(equipment_length)
exergies = np.zeros(equipment_length)
w_comp = np.zeros(equipment_length)
cost_comp = np.zeros(equipment_length)
w_tur = np.zeros(equipment_length)
cost_tur = np.zeros(equipment_length)
q_cooler = np.zeros(equipment_length)
cost_cooler = np.zeros(equipment_length)
dissipation = np.zeros(equipment_length)
q_heater = np.zeros(equipment_length)
cost_heater = np.zeros(equipment_length)
q_hx = np.zeros(equipment_length)
cost_hx = np.zeros(equipment_length)
cost_hx2= np.zeros(equipment_length)

(
    Pressures,
    Temperatures,
    approach_temp,
    approach_temp_2,
    split_ratio,
    split_ratio_2,
    mass_flow,
) = decision_variable_placement(x, enumerated_equipment, equipment_length)
Pressures = Pressure_calculation(
        Pressures, equipment, cooler_pdrop, heater_pdrop, hx_pdrop, splitter,splitter2
    )
tur_pratio, comp_pratio = tur_comp_pratio(
    enumerated_equipment, Pressures, equipment_length
)

cooler_position = [i for i, j in enumerated_equipment if j == 2]
for index in cooler_position:
    enthalpies[index], entropies[index] = enthalpy_entropy(
        Temperatures[index], Pressures[index]
    )

heater_position = [i for i, j in enumerated_equipment if j == 4]
for index in heater_position:
    enthalpies[index], entropies[index] = enthalpy_entropy(
        Temperatures[index], Pressures[index]
    )
hx_position = np.where(equipment_array==5)[0]
hx2_position =np.where(equipment_array==6)[0]
while_counter = 0
while Temperatures.prod() == 0:
    # restructuring this part can be useful, separating splitter information from tur/comp calculation while adding if checks
    # combinnig two power checks within the if check
    (
        Temperatures,
        enthalpies,
        entropies,
        w_tur,
        w_comp,
    ) = turbine_compressor_calculation(
        Temperatures,
        Pressures,
        enthalpies,
        entropies,
        w_tur,
        w_comp,
        equipment,
        ntur,
        ncomp,
        mass_flow,
    )

    if splitter == True:
        (Temperatures, enthalpies, entropies) = splitter_mixer_calc(
            Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
        )
    if splitter2 == True:
        (Temperatures, enthalpies, entropies) = splitter_mixer_calc_2(
            Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
        )


    if (
        hx_position.size != 0
        and Temperatures[hx_position[0] - 1] != 0
        and Temperatures[hx_position[1] - 1] != 0
    ):
        hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
        if (
            Temperatures[hotside_index - 1]
            < Temperatures[coldside_index - 1] + approach_temp
        ):
            print("Infeasible HX1.1")
            c= PENALTY_VALUE
        try:
            (
                Temperatures[hotside_index],
                enthalpies[hotside_index],
                entropies[hotside_index],
                Temperatures[coldside_index],
                enthalpies[coldside_index],
                entropies[coldside_index],
                q_hx[min(hotside_index, coldside_index)],
            ) = HX_calculation(
                Temperatures[hotside_index - 1],
                Pressures[hotside_index - 1],
                enthalpies[hotside_index - 1],
                Temperatures[coldside_index - 1],
                Pressures[coldside_index - 1],
                enthalpies[coldside_index - 1],
                approach_temp,
                hx_pdrop,
                mass_flow[hotside_index],
                mass_flow[coldside_index],
            )
        except:
            print("HX1 calculation error")
            c= PENALTY_VALUE


    if (
        hx2_position.size != 0
        and Temperatures[hx2_position[0] - 1] != 0
        and Temperatures[hx2_position[1] - 1] != 0
    ):
        hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
        if (
            Temperatures[hotside_index2 - 1]
            < Temperatures[coldside_index2 - 1] + approach_temp_2
        ):
            print("Infeasible HX2.1")
            c= PENALTY_VALUE
        try:
            (
                Temperatures[hotside_index2],
                enthalpies[hotside_index2],
                entropies[hotside_index2],
                Temperatures[coldside_index2],
                enthalpies[coldside_index2],
                entropies[coldside_index2],
                q_hx[min(hotside_index2, coldside_index2)],
            ) = HX_calculation(
                Temperatures[hotside_index2 - 1],
                Pressures[hotside_index2 - 1],
                enthalpies[hotside_index2 - 1],
                Temperatures[coldside_index2 - 1],
                Pressures[coldside_index2 - 1],
                enthalpies[coldside_index2 - 1],
                approach_temp,
                hx_pdrop,
                mass_flow[hotside_index2],
                mass_flow[coldside_index2],
            )
        except:
            print("HX2 calculation error")
            c= PENALTY_VALUE
    if while_counter == 5:
        print("Infeasible Temperatures")
        break
    while_counter += 1
Temperatures_old = Temperatures.copy()
Temperatures[hx_position-1]
assumed_inlet = hx_position[np.where(Temperatures[hx_position-1]==0)[0].item()]-1
converged = False
while converged == False:
    hx_converged = False
    Temperatures[assumed_inlet] = np.random.uniform(50,500)
    assumed_temperature = Temperatures[assumed_inlet].copy()
    print("Temperature Assumption",assumed_temperature)
    enthalpies[assumed_inlet], entropies[assumed_inlet] = enthalpy_entropy(Temperatures[assumed_inlet], Pressures[assumed_inlet])
    hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
    Temperatures = Temperatures_old.copy()
    if (
        Temperatures[hotside_index - 1]
        < Temperatures[coldside_index - 1] + approach_temp
    ):
        print("Infeasible HX1.1")
    else:
        try:
            (
                Temperatures[hotside_index],
                enthalpies[hotside_index],
                entropies[hotside_index],
                Temperatures[coldside_index],
                enthalpies[coldside_index],
                entropies[coldside_index],
                q_hx[min(hotside_index, coldside_index)],
            ) = HX_calculation(
                Temperatures[hotside_index - 1],
                Pressures[hotside_index - 1],
                enthalpies[hotside_index - 1],
                Temperatures[coldside_index - 1],
                Pressures[coldside_index - 1],
                enthalpies[coldside_index - 1],
                approach_temp,
                hx_pdrop,
                mass_flow[hotside_index],
                mass_flow[coldside_index],
            )
            hx_converged = True
        except:
            hx_converged = False
        if hx_converged == True:
            for while_counter2nd in range(20):
                for i in range(len(Temperatures)):
                    if Temperatures[i] != 0:
                        if i == len(Temperatures) - 1:
                                if equipment[0] == 1:
                                    (
                                        enthalpies[0],
                                        entropies[0],
                                        Temperatures[0],
                                        w_tur[0],
                                    ) = turbine(
                                        Temperatures[i],
                                        Pressures[i],
                                        Pressures[0],
                                        ntur,
                                        mass_flow[0],
                                    )
                                elif equipment[0] == 3:
                                    (
                                        enthalpies[0],
                                        entropies[0],
                                        Temperatures[0],
                                        w_comp[0],
                                    ) = compressor(
                                        Temperatures[i],
                                        Pressures[i],
                                        Pressures[0],
                                        ncomp,
                                        mass_flow[0],
                                    )
                                elif equipment[0] == 9:
                                    enthalpies[0] = enthalpies[i]
                                    entropies[0] = entropies[i]
                                    Temperatures[0] = Temperatures[i]

                        else:
                                if equipment[i + 1] == 1:
                                    (
                                        enthalpies[i + 1],
                                        entropies[i + 1],
                                        Temperatures[i + 1],
                                        w_tur[i + 1],
                                    ) = turbine(
                                        Temperatures[i],
                                        Pressures[i],
                                        Pressures[i + 1],
                                        ntur,
                                        mass_flow[i + 1],
                                    )
                                elif equipment[i + 1] == 3:
                                    (
                                        enthalpies[i + 1],
                                        entropies[i + 1],
                                        Temperatures[i + 1],
                                        w_comp[i + 1],
                                    ) = compressor(
                                        Temperatures[i],
                                        Pressures[i],
                                        Pressures[i + 1],
                                        ncomp,
                                        mass_flow[i + 1],
                                    )
                if splitter == True:
                    splitter = np.where(equipment_array == 9)[0]
                    Temperatures[splitter] = Temperatures[splitter - 1]
                    enthalpies[splitter] = enthalpies[splitter - 1]
                    entropies[splitter] = entropies[splitter - 1]
                    mixer1, mixer2 = np.where(equipment_array == 7)[0]
                    Temperatures[mixer1] = Temperatures[splitter]
                    enthalpies[mixer1] = enthalpies[splitter]
                    entropies[mixer1] = entropies[splitter]
                    if (
                        Pressures[mixer1 - 1] == Pressures[mixer2 - 1]
                        and Temperatures[mixer1 - 1] != 0
                        and Temperatures[mixer2 - 1] != 0
                    ):
                        inlet1 = Fluid(FluidsList.CarbonDioxide).with_state(
                            Input.temperature(Temperatures[mixer1 - 1]),
                            Input.pressure(Pressures[mixer1 - 1]),
                        )
                        inlet2 = Fluid(FluidsList.CarbonDioxide).with_state(
                            Input.temperature(Temperatures[mixer2 - 1]),
                            Input.pressure(Pressures[mixer2 - 1]),
                        )
                        outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                            mass_flow[mixer1 - 1], inlet1, mass_flow[mixer2 - 1], inlet2
                        )
                        Temperatures[mixer2] = outlet.temperature
                        enthalpies[mixer2] = outlet.enthalpy
                        entropies[mixer2] = outlet.entropy
                    if (
                        Temperatures[mixer1 - 1] != 0
                        and Temperatures[mixer2 - 1] != 0
                    ):
                        if Pressures[mixer1 - 1] > Pressures[mixer2 - 1]:
                            hp_index = mixer1 - 1
                            lp_index = mixer2 - 1
                        elif Pressures[mixer1 - 1] < Pressures[mixer2 - 1]:
                            hp_index = mixer2 - 1
                            lp_index = mixer1 - 1

                        hp_inlet = (
                            Fluid(FluidsList.CarbonDioxide)
                            .with_state(
                                Input.temperature(Temperatures[hp_index]),
                                Input.pressure(Pressures[hp_index]),
                            )
                            .isenthalpic_expansion_to_pressure(Pressures[lp_index])
                        )
                        lp_inlet = Fluid(FluidsList.CarbonDioxide).with_state(
                            Input.temperature(Temperatures[lp_index]),
                            Input.pressure(Pressures[lp_index]),
                        )
                        outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                            mass_flow[hp_index], hp_inlet, mass_flow[lp_index], lp_inlet
                        )
                        Temperatures[mixer2] = outlet.temperature
                        enthalpies[mixer2] = outlet.enthalpy
                        entropies[mixer2] = outlet.entropy
                if splitter2 == True:
                    splitter = np.where(equipment_array == 10)[0]
                    Temperatures[splitter] = Temperatures[splitter - 1]
                    enthalpies[splitter] = enthalpies[splitter - 1]
                    entropies[splitter] = entropies[splitter - 1]
                    mixer1, mixer2 = np.where(equipment_array == 8)[0]
                    Temperatures[mixer1] = Temperatures[splitter]
                    enthalpies[mixer1] = enthalpies[splitter]
                    entropies[mixer1] = entropies[splitter]
                    if (
                        Pressures[mixer1 - 1] == Pressures[mixer2 - 1]
                        and Temperatures[mixer1 - 1] != 0
                        and Temperatures[mixer2 - 1] != 0
                    ):
                        inlet1 = Fluid(FluidsList.CarbonDioxide).with_state(
                            Input.temperature(Temperatures[mixer1 - 1]),
                            Input.pressure(Pressures[mixer1 - 1]),
                        )
                        inlet2 = Fluid(FluidsList.CarbonDioxide).with_state(
                            Input.temperature(Temperatures[mixer2 - 1]),
                            Input.pressure(Pressures[mixer2 - 1]),
                        )
                        outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                            mass_flow[mixer1 - 1], inlet1, mass_flow[mixer2 - 1], inlet2
                        )
                        Temperatures[mixer2] = outlet.temperature
                        enthalpies[mixer2] = outlet.enthalpy
                        entropies[mixer2] = outlet.entropy
                    if (
                        Temperatures[mixer1 - 1] != 0
                        and Temperatures[mixer2 - 1] != 0
                    ):
                        if Pressures[mixer1 - 1] > Pressures[mixer2 - 1]:
                            hp_index = mixer1 - 1
                            lp_index = mixer2 - 1
                        elif Pressures[mixer1 - 1] < Pressures[mixer2 - 1]:
                            hp_index = mixer2 - 1
                            lp_index = mixer1 - 1

                        hp_inlet = (
                            Fluid(FluidsList.CarbonDioxide)
                            .with_state(
                                Input.temperature(Temperatures[hp_index]),
                                Input.pressure(Pressures[hp_index]),
                            )
                            .isenthalpic_expansion_to_pressure(Pressures[lp_index])
                        )
                        lp_inlet = Fluid(FluidsList.CarbonDioxide).with_state(
                            Input.temperature(Temperatures[lp_index]),
                            Input.pressure(Pressures[lp_index]),
                        )
                        outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                            mass_flow[hp_index], hp_inlet, mass_flow[lp_index], lp_inlet
                        )
                        Temperatures[mixer2] = outlet.temperature
                        enthalpies[mixer2] = outlet.enthalpy
                        entropies[mixer2] = outlet.entropy
                if (
                    hx_position.size != 0
                    and Temperatures[hx_position[0] - 1] != 0
                    and Temperatures[hx_position[1] - 1] != 0
                ):
                    hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
                    if (
                        Temperatures[hotside_index - 1]
                        < Temperatures[coldside_index - 1] + approach_temp
                    ):
                        print("Infeasible HX1.1")
                        c= PENALTY_VALUE
                    try:
                        (
                            Temperatures[hotside_index],
                            enthalpies[hotside_index],
                            entropies[hotside_index],
                            Temperatures[coldside_index],
                            enthalpies[coldside_index],
                            entropies[coldside_index],
                            q_hx[min(hotside_index, coldside_index)],
                        ) = HX_calculation(
                            Temperatures[hotside_index - 1],
                            Pressures[hotside_index - 1],
                            enthalpies[hotside_index - 1],
                            Temperatures[coldside_index - 1],
                            Pressures[coldside_index - 1],
                            enthalpies[coldside_index - 1],
                            approach_temp,
                            hx_pdrop,
                            mass_flow[hotside_index],
                            mass_flow[coldside_index],
                        )
                    except:
                        print("HX1 calculation error")
                        c= PENALTY_VALUE
                if (
                    hx2_position.size != 0
                    and Temperatures[hx2_position[0] - 1] != 0
                    and Temperatures[hx2_position[1] - 1] != 0
                ):
                    hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
                    if (
                        Temperatures[hotside_index2 - 1]
                        < Temperatures[coldside_index2 - 1] + approach_temp_2
                    ):
                        print("Infeasible HX2.1")
                        c= PENALTY_VALUE
                    try:
                        (
                            Temperatures[hotside_index2],
                            enthalpies[hotside_index2],
                            entropies[hotside_index2],
                            Temperatures[coldside_index2],
                            enthalpies[coldside_index2],
                            entropies[coldside_index2],
                            q_hx[min(hotside_index2, coldside_index2)],
                        ) = HX_calculation(
                            Temperatures[hotside_index2 - 1],
                            Pressures[hotside_index2 - 1],
                            enthalpies[hotside_index2 - 1],
                            Temperatures[coldside_index2 - 1],
                            Pressures[coldside_index2 - 1],
                            enthalpies[coldside_index2 - 1],
                            approach_temp,
                            hx_pdrop,
                            mass_flow[hotside_index2],
                            mass_flow[coldside_index2],
                        )
                    except:
                        print("HX2 calculation error")
                        c= PENALTY_VALUE
                if np.round(Temperatures[assumed_inlet]) != np.round(assumed_temperature):
                    Temperatures[assumed_inlet] = (Temperatures[assumed_inlet] + assumed_temperature) / 2
                    enthalpies[assumed_inlet], entropies[assumed_inlet] = enthalpy_entropy(Temperatures[assumed_inlet], Pressures[assumed_inlet])
                    assumed_temperature = Temperatures[assumed_inlet].copy()
                    print("Updated Temperature Assumption",Temperatures[assumed_inlet])
                else:
                    converged = True
                    break
for index in cooler_position:
    if Temperatures[index] >= Temperatures[index - 1]:
        # print("Infeasible Cooler")
        c= PENALTY_VALUE
enthalpies, entropies, q_cooler = cooler_calculation(
    cooler_position,
    Temperatures,
    Pressures,
    enthalpies,
    entropies,
    q_cooler,
    cooler_pdrop,
    mass_flow,
)

for index in heater_position:
    if Temperatures[index] <= Temperatures[index - 1]:
        # print("Infeasible Temperatures for heater")
        c= PENALTY_VALUE
enthalpies, entropies, q_heater = heater_calculation(
    heater_position,
    Temperatures,
    Pressures,
    enthalpies,
    entropies,
    q_heater,
    heater_pdrop,
    mass_flow,
)

total_heat = sum(q_heater)
fg_tout = fg_calculation(fg_m, total_heat)
cost_tur = turbine_econ(w_tur, Temperatures, cost_tur)
cost_comp = comp_econ(w_comp, cost_comp)
cost_cooler = cooler_econ(q_cooler, Temperatures, cost_cooler)
if np.all(cost_cooler == 0):
    c= PENALTY_VALUE
try:
    cost_heater, fg_mlist, fg_tinlist, fg_toutlist, descending_temp = heater_econ(
        equipment,
        q_heater,
        Temperatures,
        cost_heater,
        heater_position,
        total_heat,
        fg_m,
        fg_tin,
    )
except:
    # print("Heater calculation error")
    c= PENALTY_VALUE
if hx_position != []:
    cost_hx = hx_econ(q_hx, Temperatures, cost_hx, hotside_index, coldside_index)
    if np.all(cost_hx == 0):
        c= PENALTY_VALUE
if hx2_position != []:
    cost_hx2 = hx_econ(q_hx, Temperatures, cost_hx, hotside_index2, coldside_index2)
    if np.all(cost_hx == 0):
        c= PENALTY_VALUE
pec = cost_tur + cost_hx + cost_cooler + cost_comp + cost_heater+cost_hx2
prod_capacity = (sum(w_tur) - sum(w_comp)) / 1e6
zk, cfueltot, lcoe = economics(pec, prod_capacity)
for i, j, k, l in zip(equipment, Temperatures, mass_flow, pec):
        print(f" {i}, {j:.2f}, {k:.0f}, Cost: {l:.0f}")

In [ ]:
Temperatures[hx_position-1]
assumed_inlet = hx_position[np.where(Temperatures[hx_position-1]==0)[0].item()]-1
converged = False
while converged == False:
    hx_converged = False
    Temperatures[assumed_inlet] = np.random.uniform(50,500)
    assumed_temperature = Temperatures[assumed_inlet].copy()
    print("Temperature Assumption",assumed_temperature)
    enthalpies[assumed_inlet], entropies[assumed_inlet] = enthalpy_entropy(Temperatures[assumed_inlet], Pressures[assumed_inlet])
    hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
    Temperatures = Temperatures_old.copy()
    if (
        Temperatures[hotside_index - 1]
        < Temperatures[coldside_index - 1] + approach_temp
    ):
        print("Infeasible HX1.1")
    else:
        try:
            (
                Temperatures[hotside_index],
                enthalpies[hotside_index],
                entropies[hotside_index],
                Temperatures[coldside_index],
                enthalpies[coldside_index],
                entropies[coldside_index],
                q_hx[min(hotside_index, coldside_index)],
            ) = HX_calculation(
                Temperatures[hotside_index - 1],
                Pressures[hotside_index - 1],
                enthalpies[hotside_index - 1],
                Temperatures[coldside_index - 1],
                Pressures[coldside_index - 1],
                enthalpies[coldside_index - 1],
                approach_temp,
                hx_pdrop,
                mass_flow[hotside_index],
                mass_flow[coldside_index],
            )
            hx_converged = True
        except:
            hx_converged = False
        if hx_converged == True:
            for while_counter2nd in range(20):
                for i in range(len(Temperatures)):
                    if Temperatures[i] != 0:
                        if i == len(Temperatures) - 1:
                                if equipment[0] == 1:
                                    (
                                        enthalpies[0],
                                        entropies[0],
                                        Temperatures[0],
                                        w_tur[0],
                                    ) = turbine(
                                        Temperatures[i],
                                        Pressures[i],
                                        Pressures[0],
                                        ntur,
                                        mass_flow[0],
                                    )
                                elif equipment[0] == 3:
                                    (
                                        enthalpies[0],
                                        entropies[0],
                                        Temperatures[0],
                                        w_comp[0],
                                    ) = compressor(
                                        Temperatures[i],
                                        Pressures[i],
                                        Pressures[0],
                                        ncomp,
                                        mass_flow[0],
                                    )
                                elif equipment[0] == 9:
                                    enthalpies[0] = enthalpies[i]
                                    entropies[0] = entropies[i]
                                    Temperatures[0] = Temperatures[i]

                        else:
                                if equipment[i + 1] == 1:
                                    (
                                        enthalpies[i + 1],
                                        entropies[i + 1],
                                        Temperatures[i + 1],
                                        w_tur[i + 1],
                                    ) = turbine(
                                        Temperatures[i],
                                        Pressures[i],
                                        Pressures[i + 1],
                                        ntur,
                                        mass_flow[i + 1],
                                    )
                                elif equipment[i + 1] == 3:
                                    (
                                        enthalpies[i + 1],
                                        entropies[i + 1],
                                        Temperatures[i + 1],
                                        w_comp[i + 1],
                                    ) = compressor(
                                        Temperatures[i],
                                        Pressures[i],
                                        Pressures[i + 1],
                                        ncomp,
                                        mass_flow[i + 1],
                                    )
                if splitter == True:
                    splitter = np.where(equipment_array == 9)[0]
                    Temperatures[splitter] = Temperatures[splitter - 1]
                    enthalpies[splitter] = enthalpies[splitter - 1]
                    entropies[splitter] = entropies[splitter - 1]
                    mixer1, mixer2 = np.where(equipment_array == 7)[0]
                    Temperatures[mixer1] = Temperatures[splitter]
                    enthalpies[mixer1] = enthalpies[splitter]
                    entropies[mixer1] = entropies[splitter]
                    if (
                        Pressures[mixer1 - 1] == Pressures[mixer2 - 1]
                        and Temperatures[mixer1 - 1] != 0
                        and Temperatures[mixer2 - 1] != 0
                    ):
                        inlet1 = Fluid(FluidsList.CarbonDioxide).with_state(
                            Input.temperature(Temperatures[mixer1 - 1]),
                            Input.pressure(Pressures[mixer1 - 1]),
                        )
                        inlet2 = Fluid(FluidsList.CarbonDioxide).with_state(
                            Input.temperature(Temperatures[mixer2 - 1]),
                            Input.pressure(Pressures[mixer2 - 1]),
                        )
                        outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                            mass_flow[mixer1 - 1], inlet1, mass_flow[mixer2 - 1], inlet2
                        )
                        Temperatures[mixer2] = outlet.temperature
                        enthalpies[mixer2] = outlet.enthalpy
                        entropies[mixer2] = outlet.entropy
                    if (
                        Temperatures[mixer1 - 1] != 0
                        and Temperatures[mixer2 - 1] != 0
                    ):
                        if Pressures[mixer1 - 1] > Pressures[mixer2 - 1]:
                            hp_index = mixer1 - 1
                            lp_index = mixer2 - 1
                        elif Pressures[mixer1 - 1] < Pressures[mixer2 - 1]:
                            hp_index = mixer2 - 1
                            lp_index = mixer1 - 1

                        hp_inlet = (
                            Fluid(FluidsList.CarbonDioxide)
                            .with_state(
                                Input.temperature(Temperatures[hp_index]),
                                Input.pressure(Pressures[hp_index]),
                            )
                            .isenthalpic_expansion_to_pressure(Pressures[lp_index])
                        )
                        lp_inlet = Fluid(FluidsList.CarbonDioxide).with_state(
                            Input.temperature(Temperatures[lp_index]),
                            Input.pressure(Pressures[lp_index]),
                        )
                        outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                            mass_flow[hp_index], hp_inlet, mass_flow[lp_index], lp_inlet
                        )
                        Temperatures[mixer2] = outlet.temperature
                        enthalpies[mixer2] = outlet.enthalpy
                        entropies[mixer2] = outlet.entropy
                if splitter2 == True:
                    splitter = np.where(equipment_array == 10)[0]
                    Temperatures[splitter] = Temperatures[splitter - 1]
                    enthalpies[splitter] = enthalpies[splitter - 1]
                    entropies[splitter] = entropies[splitter - 1]
                    mixer1, mixer2 = np.where(equipment_array == 8)[0]
                    Temperatures[mixer1] = Temperatures[splitter]
                    enthalpies[mixer1] = enthalpies[splitter]
                    entropies[mixer1] = entropies[splitter]
                    if (
                        Pressures[mixer1 - 1] == Pressures[mixer2 - 1]
                        and Temperatures[mixer1 - 1] != 0
                        and Temperatures[mixer2 - 1] != 0
                    ):
                        inlet1 = Fluid(FluidsList.CarbonDioxide).with_state(
                            Input.temperature(Temperatures[mixer1 - 1]),
                            Input.pressure(Pressures[mixer1 - 1]),
                        )
                        inlet2 = Fluid(FluidsList.CarbonDioxide).with_state(
                            Input.temperature(Temperatures[mixer2 - 1]),
                            Input.pressure(Pressures[mixer2 - 1]),
                        )
                        outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                            mass_flow[mixer1 - 1], inlet1, mass_flow[mixer2 - 1], inlet2
                        )
                        Temperatures[mixer2] = outlet.temperature
                        enthalpies[mixer2] = outlet.enthalpy
                        entropies[mixer2] = outlet.entropy
                    if (
                        Temperatures[mixer1 - 1] != 0
                        and Temperatures[mixer2 - 1] != 0
                    ):
                        if Pressures[mixer1 - 1] > Pressures[mixer2 - 1]:
                            hp_index = mixer1 - 1
                            lp_index = mixer2 - 1
                        elif Pressures[mixer1 - 1] < Pressures[mixer2 - 1]:
                            hp_index = mixer2 - 1
                            lp_index = mixer1 - 1

                        hp_inlet = (
                            Fluid(FluidsList.CarbonDioxide)
                            .with_state(
                                Input.temperature(Temperatures[hp_index]),
                                Input.pressure(Pressures[hp_index]),
                            )
                            .isenthalpic_expansion_to_pressure(Pressures[lp_index])
                        )
                        lp_inlet = Fluid(FluidsList.CarbonDioxide).with_state(
                            Input.temperature(Temperatures[lp_index]),
                            Input.pressure(Pressures[lp_index]),
                        )
                        outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                            mass_flow[hp_index], hp_inlet, mass_flow[lp_index], lp_inlet
                        )
                        Temperatures[mixer2] = outlet.temperature
                        enthalpies[mixer2] = outlet.enthalpy
                        entropies[mixer2] = outlet.entropy
                if (
                    hx_position.size != 0
                    and Temperatures[hx_position[0] - 1] != 0
                    and Temperatures[hx_position[1] - 1] != 0
                ):
                    hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
                    if (
                        Temperatures[hotside_index - 1]
                        < Temperatures[coldside_index - 1] + approach_temp
                    ):
                        print("Infeasible HX1.1")
                        c= PENALTY_VALUE
                    try:
                        (
                            Temperatures[hotside_index],
                            enthalpies[hotside_index],
                            entropies[hotside_index],
                            Temperatures[coldside_index],
                            enthalpies[coldside_index],
                            entropies[coldside_index],
                            q_hx[min(hotside_index, coldside_index)],
                        ) = HX_calculation(
                            Temperatures[hotside_index - 1],
                            Pressures[hotside_index - 1],
                            enthalpies[hotside_index - 1],
                            Temperatures[coldside_index - 1],
                            Pressures[coldside_index - 1],
                            enthalpies[coldside_index - 1],
                            approach_temp,
                            hx_pdrop,
                            mass_flow[hotside_index],
                            mass_flow[coldside_index],
                        )
                    except:
                        print("HX1 calculation error")
                        c= PENALTY_VALUE
                if (
                    hx2_position.size != 0
                    and Temperatures[hx2_position[0] - 1] != 0
                    and Temperatures[hx2_position[1] - 1] != 0
                ):
                    hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
                    if (
                        Temperatures[hotside_index2 - 1]
                        < Temperatures[coldside_index2 - 1] + approach_temp_2
                    ):
                        print("Infeasible HX2.1")
                        c= PENALTY_VALUE
                    try:
                        (
                            Temperatures[hotside_index2],
                            enthalpies[hotside_index2],
                            entropies[hotside_index2],
                            Temperatures[coldside_index2],
                            enthalpies[coldside_index2],
                            entropies[coldside_index2],
                            q_hx[min(hotside_index2, coldside_index2)],
                        ) = HX_calculation(
                            Temperatures[hotside_index2 - 1],
                            Pressures[hotside_index2 - 1],
                            enthalpies[hotside_index2 - 1],
                            Temperatures[coldside_index2 - 1],
                            Pressures[coldside_index2 - 1],
                            enthalpies[coldside_index2 - 1],
                            approach_temp,
                            hx_pdrop,
                            mass_flow[hotside_index2],
                            mass_flow[coldside_index2],
                        )
                    except:
                        print("HX2 calculation error")
                        c= PENALTY_VALUE
                if np.round(Temperatures[assumed_inlet]) != np.round(assumed_temperature):
                    Temperatures[assumed_inlet] = (Temperatures[assumed_inlet] + assumed_temperature) / 2
                    enthalpies[assumed_inlet], entropies[assumed_inlet] = enthalpy_entropy(Temperatures[assumed_inlet], Pressures[assumed_inlet])
                    assumed_temperature = Temperatures[assumed_inlet].copy()
                    print("Updated Temperature Assumption",Temperatures[assumed_inlet])
                else:
                    converged = True
                    break
print(converged)

In [ ]:
print(Temperatures)
print(Temperatures_old)
objective_temperatures = np.array([343.6,451.8,570.8,465.9,343.6,756.9,636.7,465.9,465.9,368.8,305.4,343.6])
print(objective_temperatures-273.15-Temperatures)
assumed_temperature, Temperatures[assumed_inlet]

In [ ]:


def results_analysis(x, equipment):
    ntur = 85  # 2019 Nabil 93
    ncomp = 82  #  89
    fg_tin = 539.76  # °C 630
    fg_m = 68.75  # kg/s 935
    cooler_pdrop = 1e5  # 0.5e5
    heater_pdrop = 0  # 1e5
    hx_pdrop = 0.5e5  # 1e5
    PENALTY_VALUE = float(1e6)
    splitter = False
    if 9 in equipment:
        splitter = True
    enumerated_equipment = list(enumerate(equipment))
    equipment_length = len(equipment)
    enthalpies = np.zeros(equipment_length)
    entropies = np.zeros(equipment_length)
    exergies = np.zeros(equipment_length)
    w_comp = np.zeros(equipment_length)
    cost_comp = np.zeros(equipment_length)
    w_tur = np.zeros(equipment_length)
    cost_tur = np.zeros(equipment_length)
    q_cooler = np.zeros(equipment_length)
    cost_cooler = np.zeros(equipment_length)
    dissipation = np.zeros(equipment_length)
    q_heater = np.zeros(equipment_length)
    cost_heater = np.zeros(equipment_length)
    q_hx = np.zeros(equipment_length)
    cost_hx = np.zeros(equipment_length)

    (
        Pressures,
        Temperatures,
        approach_temp,
        split_ratio,
        mass_flow,
    ) = decision_variable_placement(x, enumerated_equipment, equipment_length)

    Pressures = Pressure_calculation(
        Pressures, equipment, cooler_pdrop, heater_pdrop, hx_pdrop, splitter
    )

    # it can benefit from tur_ppisition and comp_position
    # Turbine and Compressor pressure ratio calculation and checking
    tur_pratio, comp_pratio = tur_comp_pratio(
        enumerated_equipment, Pressures, equipment_length
    )

    cooler_position = [i for i, j in enumerated_equipment if j == 2]
    for index in cooler_position:
        enthalpies[index], entropies[index] = enthalpy_entropy(
            Temperatures[index], Pressures[index]
        )

    heater_position = [i for i, j in enumerated_equipment if j == 4]
    for index in heater_position:
        enthalpies[index], entropies[index] = enthalpy_entropy(
            Temperatures[index], Pressures[index]
        )

    while_counter = 0
    while Temperatures.prod() == 0:
        # restructuring this part can be useful, separating splitter information from tur/comp calculation while adding if checks
        # combinnig two power checks within the if check
        (
            Temperatures,
            enthalpies,
            entropies,
            w_tur,
            w_comp,
        ) = turbine_compressor_calculation(
            Temperatures,
            Pressures,
            enthalpies,
            entropies,
            w_tur,
            w_comp,
            equipment,
            ntur,
            ncomp,
            mass_flow,
        )

        if splitter == True:
            (Temperatures, enthalpies, entropies) = splitter_mixer_calc(
                Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
            )

        hx_position = [i for i, j in enumerated_equipment if j == 5]
        if (
            hx_position != []
            and Temperatures[hx_position[0] - 1] != 0
            and Temperatures[hx_position[1] - 1] != 0
        ):
            hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
            if (
                Temperatures[hotside_index - 1]
                < Temperatures[coldside_index - 1] + approach_temp
            ):
                # print("Infeasible HX")
                return PENALTY_VALUE
            if (
                mass_flow[hotside_index - 1] * enthalpies[hotside_index - 1]
                < mass_flow[coldside_index - 1] * enthalpies[coldside_index - 1]
            ):
                # print("Infeasible HX")
                return PENALTY_VALUE
            try:
                (
                    Temperatures[hotside_index],
                    enthalpies[hotside_index],
                    entropies[hotside_index],
                    Temperatures[coldside_index],
                    enthalpies[coldside_index],
                    entropies[coldside_index],
                    q_hx[min(hotside_index, coldside_index)],
                ) = HX_calculation(
                    Temperatures[hotside_index - 1],
                    Pressures[hotside_index - 1],
                    enthalpies[hotside_index - 1],
                    Temperatures[coldside_index - 1],
                    Pressures[coldside_index - 1],
                    enthalpies[coldside_index - 1],
                    approach_temp,
                    hx_pdrop,
                    mass_flow[hotside_index],
                    mass_flow[coldside_index],
                )
            except:
                # print("HX calculation error")
                return PENALTY_VALUE
        if while_counter == 3:
            # print("Infeasible Temperatures")
            return PENALTY_VALUE
        while_counter += 1

    for index in cooler_position:
        if Temperatures[index] >= Temperatures[index - 1]:
            # print("Infeasible Cooler")
            return PENALTY_VALUE
    enthalpies, entropies, q_cooler = cooler_calculation(
        cooler_position,
        Temperatures,
        Pressures,
        enthalpies,
        entropies,
        q_cooler,
        cooler_pdrop,
        mass_flow,
    )

    for index in heater_position:
        if Temperatures[index] <= Temperatures[index - 1]:
            # print("Infeasible Temperatures for heater")
            return PENALTY_VALUE
    enthalpies, entropies, q_heater = heater_calculation(
        heater_position,
        Temperatures,
        Pressures,
        enthalpies,
        entropies,
        q_heater,
        heater_pdrop,
        mass_flow,
    )

    total_heat = sum(q_heater)
    fg_tout = fg_calculation(fg_m, total_heat)

    # Economic Analysis
    cost_tur = turbine_econ(w_tur, Temperatures, cost_tur)
    cost_comp = comp_econ(w_comp, cost_comp)
    cost_cooler = cooler_econ(q_cooler, Temperatures, cost_cooler)
    if np.all(cost_cooler == 0):
        return PENALTY_VALUE
    try:
        cost_heater, fg_mlist, fg_tinlist, fg_toutlist, descending_temp = heater_econ(
            equipment,
            q_heater,
            Temperatures,
            cost_heater,
            heater_position,
            total_heat,
            fg_m,
            fg_tin,
        )
    except:
        # print("Heater calculation error")
        return PENALTY_VALUE
    if hx_position != []:
        cost_hx = hx_econ(q_hx, Temperatures, cost_hx, hotside_index, coldside_index)
        if np.all(cost_hx == 0):
            return PENALTY_VALUE
    pec = cost_tur + cost_hx + cost_cooler + cost_comp + cost_heater
    prod_capacity = (sum(w_tur) - sum(w_comp)) / 1e6
    zk, cfueltot, lcoe = economics(pec, prod_capacity)

    # Exergy Analysis
    exergies, e_fgin, e_fgout = exergy_calculation(
        mass_flow,
        enthalpies,
        entropies,
        q_heater,
        fg_mlist,
        fg_tinlist,
        fg_toutlist,
        equipment_length,
    )
    # Thermo-economic Analysis
    if hx_position == []:
        hotside_index = 0
        coldside_index = 0
    
    m1, m2 = exergoeconomic_calculation(
        equipment,
        Temperatures,
        enumerated_equipment,
        exergies,
        w_tur,
        w_comp,
        descending_temp,
        e_fgin,
        e_fgout,
        zk,
        hotside_index,
        coldside_index,
    )
    try:
        costs = np.linalg.solve(m1, m2)
    except:
        print("Matrix solution problem")
        return PENALTY_VALUE
    Closs = costs[equipment_length + 1] * min(x for x in e_fgout if x != 0)
    Cfuel = costs[equipment_length] * FGINLETEXERGY
    Ztot = sum(zk)
    Cproduct = Cfuel + Ztot - Closs
    Ep = sum(w_tur) - sum(w_comp)
    for i, j in enumerated_equipment:
        if j == 2:
            dissipation[i] = costs[i] * (exergies[i - 1] - exergies[i]) + zk[i]
    Cdiss = sum(dissipation)
    lcoe_calculated = (costs[-1] * Ep + Cdiss + Closs) / (Ep / 1e6)
    c = lcoe_calculated
    thermal_efficiency = (Ep) / 40.53e6
    if thermal_efficiency < 0.1575:
        j = 100 * (0.30 - thermal_efficiency)
    else:
        j = c + 1 * max(0, 0.1 - sum(q_hx) / sum(q_heater))
    print(
        f"""
    Equipment = {equipment}
    Turbine Pratio = {tur_pratio[np.where(tur_pratio>1.0001)[0]]}
    Turbine Output = {w_tur[np.where(w_tur>0)[0]]/1e6}MW
    Compressor Pratio = {comp_pratio[np.where(comp_pratio>1.0001)[0]]}
    Compressor Input = {w_comp[np.where(w_comp>0)[0]]/1e6}MW
    Split Ratio = {split_ratio:.2f} mass_flow = {mass_flow[:5]}
    Temperatures = {Temperatures}   Tstack = {min(fg_toutlist[np.where(fg_toutlist>0)[0]]):.2f}    DT = {approach_temp}
    Pressures =    {Pressures/1e5} bar
    Equipment Cost = {pec/1e3}
    Equipment Duty = Qheater={q_heater[np.where(q_heater>0)[0]]/1e6}MW   Qcooler={q_cooler[np.where(q_cooler>0)[0]]/1e6}MW   Qhx={q_hx[np.where(q_hx>0)[0]]/1e6}MW
    Objective Function value = {c:.2f} LCOE = {lcoe:.2f} LCOEX = {lcoe_calculated:.2f}
    Exergy of streams = {exergies/1e6}MW
    Exergy of FG streams = {e_fgin[np.where(e_fgin>0)]/1e6}
                           {e_fgout[np.where(e_fgout>0)]/1e6}
                           {fg_mlist[np.where(e_fgout>0)]}
    Exergy costing of streams = {costs/3600*1e9} $/GJ
    Total PEC = {sum(pec):.2f} $
    Total Zk  = {sum(zk):.2f} $/h
    Cdiss = {Cdiss:.2f} Cl = {Closs:.2f} Cp ={costs[-1]*Ep:.2f} LCOE = {lcoe:.2f} LCOEX = {lcoe_calculated:.2f}
    Cp/Ep = {Cproduct/(Ep/1e6):.2f}
    Thermal efficiency = {thermal_efficiency*100:.2f}%
    Heat recuperation ratio = {sum(q_hx)/(sum(q_heater))*100:.2f}
    j = {j:.2f}
        """
    )
    return [
        sum(w_tur) / 1e6,
        sum(w_comp) / 1e6,
        (sum(w_tur) / 1e6 - sum(w_comp) / 1e6),
        sum(q_heater) / 1e6,
        sum(q_cooler) / 1e6,
        sum(q_hx) / 1e6,
        thermal_efficiency,
        sum(zk),
        lcoe,
        lcoe_calculated,
        j,
    ]


In [ ]:
# if Temperatures[hx_position-1].sum() > 0:
#     assumed_outlet = hx_position[np.where(Temperatures[hx_position-1]!=0)[0].item()]
#     real_inlet = assumed_outlet-1
#     Temperatures[assumed_outlet] = Temperatures[real_inlet]-approach_temp
#     enthalpies[assumed_outlet],entropies[assumed_outlet] = enthalpy_entropy(Temperatures[assumed_outlet],Pressures[assumed_outlet])
#     while Temperatures.prod():
#         for i in range(len(Temperatures)):
#             if Temperatures[i] != 0:
#                 if i == len(Temperatures) - 1:
#                     if Temperatures[0] == 0:
#                         if equipment[0] == 1:
#                             (
#                                 enthalpies[0],
#                                 entropies[0],
#                                 Temperatures[0],
#                                 w_tur[0],
#                             ) = turbine(
#                                 Temperatures[i],
#                                 Pressures[i],
#                                 Pressures[0],
#                                 ntur,
#                                 mass_flow[0],
#                             )
#                         elif equipment[0] == 3:
#                             (
#                                 enthalpies[0],
#                                 entropies[0],
#                                 Temperatures[0],
#                                 w_comp[0],
#                             ) = compressor(
#                                 Temperatures[i],
#                                 Pressures[i],
#                                 Pressures[0],
#                                 ncomp,
#                                 mass_flow[0],
#                             )
#                         elif equipment[0] == 9:
#                             enthalpies[0] = enthalpies[i]
#                             entropies[0] = entropies[i]
#                             Temperatures[0] = Temperatures[i]

#                 else:
#                         if equipment[i + 1] == 1:
#                             (
#                                 enthalpies[i + 1],
#                                 entropies[i + 1],
#                                 Temperatures[i + 1],
#                                 w_tur[i + 1],
#                             ) = turbine(
#                                 Temperatures[i],
#                                 Pressures[i],
#                                 Pressures[i + 1],
#                                 ntur,
#                                 mass_flow[i + 1],
#                             )
#                         elif equipment[i + 1] == 3:
#                             (
#                                 enthalpies[i + 1],
#                                 entropies[i + 1],
#                                 Temperatures[i + 1],
#                                 w_comp[i + 1],
#                             ) = compressor(
#                                 Temperatures[i],
#                                 Pressures[i],
#                                 Pressures[i + 1],
#                                 ncomp,
#                                 mass_flow[i + 1],
#                             )
        
#     if splitter == True:
#         (Temperatures, enthalpies, entropies) = splitter_mixer_calc(
#             Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
#         )
#     if splitter2 == True:
#         (Temperatures, enthalpies, entropies) = splitter_mixer_calc_2(
#             Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
#         )


#     if (
#         hx_position.size != 0
#         and Temperatures[hx_position[0] - 1] != 0
#         and Temperatures[hx_position[1] - 1] != 0
#     ):
#         hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
#         if (
#             Temperatures[hotside_index - 1]
#             < Temperatures[coldside_index - 1] + approach_temp
#         ):
#             print("Infeasible HX1.1")
#             c= PENALTY_VALUE
#         # if (
#         #     mass_flow[hotside_index - 1] * enthalpies[hotside_index - 1]
#         #     < mass_flow[coldside_index - 1] * enthalpies[coldside_index - 1]
#         # ):
#         #     print("Infeasible HX1.2")
#         #     c= PENALTY_VALUE
#         try:
#             (
#                 Temperatures[hotside_index],
#                 enthalpies[hotside_index],
#                 entropies[hotside_index],
#                 Temperatures[coldside_index],
#                 enthalpies[coldside_index],
#                 entropies[coldside_index],
#                 q_hx[min(hotside_index, coldside_index)],
#             ) = HX_calculation(
#                 Temperatures[hotside_index - 1],
#                 Pressures[hotside_index - 1],
#                 enthalpies[hotside_index - 1],
#                 Temperatures[coldside_index - 1],
#                 Pressures[coldside_index - 1],
#                 enthalpies[coldside_index - 1],
#                 approach_temp,
#                 hx_pdrop,
#                 mass_flow[hotside_index],
#                 mass_flow[coldside_index],
#             )
#         except:
#             print("HX1 calculation error")
#             c= PENALTY_VALUE


#     if (
#         hx2_position.size != 0
#         and Temperatures[hx2_position[0] - 1] != 0
#         and Temperatures[hx2_position[1] - 1] != 0
#     ):
#         hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
#         if (
#             Temperatures[hotside_index2 - 1]
#             < Temperatures[coldside_index2 - 1] + approach_temp_2
#         ):
#             print("Infeasible HX2.1")
#             c= PENALTY_VALUE
#         if (
#             mass_flow[hotside_index2 - 1] * enthalpies[hotside_index2 - 1]
#             < mass_flow[coldside_index2 - 1] * enthalpies[coldside_index2 - 1]
#         ):
#             print("Infeasible HX2.2")
#             c= PENALTY_VALUE
#         try:
#             (
#                 Temperatures[hotside_index2],
#                 enthalpies[hotside_index2],
#                 entropies[hotside_index2],
#                 Temperatures[coldside_index2],
#                 enthalpies[coldside_index2],
#                 entropies[coldside_index2],
#                 q_hx[min(hotside_index2, coldside_index2)],
#             ) = HX_calculation(
#                 Temperatures[hotside_index2 - 1],
#                 Pressures[hotside_index2 - 1],
#                 enthalpies[hotside_index2 - 1],
#                 Temperatures[coldside_index2 - 1],
#                 Pressures[coldside_index2 - 1],
#                 enthalpies[coldside_index2 - 1],
#                 approach_temp,
#                 hx_pdrop,
#                 mass_flow[hotside_index2],
#                 mass_flow[coldside_index2],
#             )
#         except:
#             print("HX2 calculation error")
#             c= PENALTY_VALUE
            
#     Temperatures[assumed_outlet] = Temperatures[real_inlet]+approach_temp
#     enthalpies[assumed_outlet],entropies[assumed_outlet] = enthalpy_entropy(Temperatures[assumed_outlet],Pressures[assumed_outlet])
#     while Temperatures.prod():
#         for i in range(len(Temperatures)):
#             if Temperatures[i] != 0:
#                 if i == len(Temperatures) - 1:
#                     if Temperatures[0] == 0:
#                         if equipment[0] == 1:
#                             (
#                                 enthalpies[0],
#                                 entropies[0],
#                                 Temperatures[0],
#                                 w_tur[0],
#                             ) = turbine(
#                                 Temperatures[i],
#                                 Pressures[i],
#                                 Pressures[0],
#                                 ntur,
#                                 mass_flow[0],
#                             )
#                         elif equipment[0] == 3:
#                             (
#                                 enthalpies[0],
#                                 entropies[0],
#                                 Temperatures[0],
#                                 w_comp[0],
#                             ) = compressor(
#                                 Temperatures[i],
#                                 Pressures[i],
#                                 Pressures[0],
#                                 ncomp,
#                                 mass_flow[0],
#                             )
#                         elif equipment[0] == 9:
#                             enthalpies[0] = enthalpies[i]
#                             entropies[0] = entropies[i]
#                             Temperatures[0] = Temperatures[i]

#                 else:
#                         if equipment[i + 1] == 1:
#                             (
#                                 enthalpies[i + 1],
#                                 entropies[i + 1],
#                                 Temperatures[i + 1],
#                                 w_tur[i + 1],
#                             ) = turbine(
#                                 Temperatures[i],
#                                 Pressures[i],
#                                 Pressures[i + 1],
#                                 ntur,
#                                 mass_flow[i + 1],
#                             )
#                         elif equipment[i + 1] == 3:
#                             (
#                                 enthalpies[i + 1],
#                                 entropies[i + 1],
#                                 Temperatures[i + 1],
#                                 w_comp[i + 1],
#                             ) = compressor(
#                                 Temperatures[i],
#                                 Pressures[i],
#                                 Pressures[i + 1],
#                                 ncomp,
#                                 mass_flow[i + 1],
#                             )
        
#     if splitter == True:
#         (Temperatures, enthalpies, entropies) = splitter_mixer_calc(
#             Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
#         )
#     if splitter2 == True:
#         (Temperatures, enthalpies, entropies) = splitter_mixer_calc_2(
#             Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
#         )


#     if (
#         hx_position.size != 0
#         and Temperatures[hx_position[0] - 1] != 0
#         and Temperatures[hx_position[1] - 1] != 0
#     ):
#         hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
#         if (
#             Temperatures[hotside_index - 1]
#             < Temperatures[coldside_index - 1] + approach_temp
#         ):
#             print("Infeasible HX1.1")
#             c= PENALTY_VALUE
#         # if (
#         #     mass_flow[hotside_index - 1] * enthalpies[hotside_index - 1]
#         #     < mass_flow[coldside_index - 1] * enthalpies[coldside_index - 1]
#         # ):
#         #     print("Infeasible HX1.2")
#         #     c= PENALTY_VALUE
#         try:
#             (
#                 Temperatures[hotside_index],
#                 enthalpies[hotside_index],
#                 entropies[hotside_index],
#                 Temperatures[coldside_index],
#                 enthalpies[coldside_index],
#                 entropies[coldside_index],
#                 q_hx[min(hotside_index, coldside_index)],
#             ) = HX_calculation(
#                 Temperatures[hotside_index - 1],
#                 Pressures[hotside_index - 1],
#                 enthalpies[hotside_index - 1],
#                 Temperatures[coldside_index - 1],
#                 Pressures[coldside_index - 1],
#                 enthalpies[coldside_index - 1],
#                 approach_temp,
#                 hx_pdrop,
#                 mass_flow[hotside_index],
#                 mass_flow[coldside_index],
#             )
#         except:
#             print("HX1 calculation error")
#             c= PENALTY_VALUE


#     if (
#         hx2_position.size != 0
#         and Temperatures[hx2_position[0] - 1] != 0
#         and Temperatures[hx2_position[1] - 1] != 0
#     ):
#         hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
#         if (
#             Temperatures[hotside_index2 - 1]
#             < Temperatures[coldside_index2 - 1] + approach_temp_2
#         ):
#             print("Infeasible HX2.1")
#             c= PENALTY_VALUE
#         if (
#             mass_flow[hotside_index2 - 1] * enthalpies[hotside_index2 - 1]
#             < mass_flow[coldside_index2 - 1] * enthalpies[coldside_index2 - 1]
#         ):
#             print("Infeasible HX2.2")
#             c= PENALTY_VALUE
#         try:
#             (
#                 Temperatures[hotside_index2],
#                 enthalpies[hotside_index2],
#                 entropies[hotside_index2],
#                 Temperatures[coldside_index2],
#                 enthalpies[coldside_index2],
#                 entropies[coldside_index2],
#                 q_hx[min(hotside_index2, coldside_index2)],
#             ) = HX_calculation(
#                 Temperatures[hotside_index2 - 1],
#                 Pressures[hotside_index2 - 1],
#                 enthalpies[hotside_index2 - 1],
#                 Temperatures[coldside_index2 - 1],
#                 Pressures[coldside_index2 - 1],
#                 enthalpies[coldside_index2 - 1],
#                 approach_temp,
#                 hx_pdrop,
#                 mass_flow[hotside_index2],
#                 mass_flow[coldside_index2],
#             )
#         except:
#             print("HX2 calculation error")
#             c= PENALTY_VALUE
#     Temperatures[assumed_outlet],enthalpies[assumed_outlet],entropies[assumed_outlet] = 0,0,0
if Temperatures[hx2_position-1].sum() > 0:
    assumed_outlet = hx2_position[np.where(Temperatures[hx2_position-1]!=0)[0].item()]
    real_inlet = assumed_outlet-1
    Temperatures[assumed_outlet] = Temperatures[real_inlet]-approach_temp
    enthalpies[assumed_outlet],entropies[assumed_outlet] = enthalpy_entropy(Temperatures[assumed_outlet],Pressures[assumed_outlet])
    while Temperatures.prod():
        for i in range(len(Temperatures)):
            if Temperatures[i] != 0:
                if i == len(Temperatures) - 1:
                    if Temperatures[0] == 0:
                        if equipment[0] == 1:
                            (
                                enthalpies[0],
                                entropies[0],
                                Temperatures[0],
                                w_tur[0],
                            ) = turbine(
                                Temperatures[i],
                                Pressures[i],
                                Pressures[0],
                                ntur,
                                mass_flow[0],
                            )
                        elif equipment[0] == 3:
                            (
                                enthalpies[0],
                                entropies[0],
                                Temperatures[0],
                                w_comp[0],
                            ) = compressor(
                                Temperatures[i],
                                Pressures[i],
                                Pressures[0],
                                ncomp,
                                mass_flow[0],
                            )
                        elif equipment[0] == 9:
                            enthalpies[0] = enthalpies[i]
                            entropies[0] = entropies[i]
                            Temperatures[0] = Temperatures[i]

                else:
                        if equipment[i + 1] == 1:
                            (
                                enthalpies[i + 1],
                                entropies[i + 1],
                                Temperatures[i + 1],
                                w_tur[i + 1],
                            ) = turbine(
                                Temperatures[i],
                                Pressures[i],
                                Pressures[i + 1],
                                ntur,
                                mass_flow[i + 1],
                            )
                        elif equipment[i + 1] == 3:
                            (
                                enthalpies[i + 1],
                                entropies[i + 1],
                                Temperatures[i + 1],
                                w_comp[i + 1],
                            ) = compressor(
                                Temperatures[i],
                                Pressures[i],
                                Pressures[i + 1],
                                ncomp,
                                mass_flow[i + 1],
                            )
        
    if splitter == True:
        (Temperatures, enthalpies, entropies) = splitter_mixer_calc(
            Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
        )
    if splitter2 == True:
        (Temperatures, enthalpies, entropies) = splitter_mixer_calc_2(
            Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
        )


    if (
        hx_position.size != 0
        and Temperatures[hx_position[0] - 1] != 0
        and Temperatures[hx_position[1] - 1] != 0
    ):
        hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
        if (
            Temperatures[hotside_index - 1]
            < Temperatures[coldside_index - 1] + approach_temp
        ):
            print("Infeasible HX1.1")
            c= PENALTY_VALUE
        # if (
        #     mass_flow[hotside_index - 1] * enthalpies[hotside_index - 1]
        #     < mass_flow[coldside_index - 1] * enthalpies[coldside_index - 1]
        # ):
        #     print("Infeasible HX1.2")
        #     c= PENALTY_VALUE
        try:
            (
                Temperatures[hotside_index],
                enthalpies[hotside_index],
                entropies[hotside_index],
                Temperatures[coldside_index],
                enthalpies[coldside_index],
                entropies[coldside_index],
                q_hx[min(hotside_index, coldside_index)],
            ) = HX_calculation(
                Temperatures[hotside_index - 1],
                Pressures[hotside_index - 1],
                enthalpies[hotside_index - 1],
                Temperatures[coldside_index - 1],
                Pressures[coldside_index - 1],
                enthalpies[coldside_index - 1],
                approach_temp,
                hx_pdrop,
                mass_flow[hotside_index],
                mass_flow[coldside_index],
            )
        except:
            print("HX1 calculation error")
            c= PENALTY_VALUE


    if (
        hx2_position.size != 0
        and Temperatures[hx2_position[0] - 1] != 0
        and Temperatures[hx2_position[1] - 1] != 0
    ):
        hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
        if (
            Temperatures[hotside_index2 - 1]
            < Temperatures[coldside_index2 - 1] + approach_temp_2
        ):
            print("Infeasible HX2.1")
            c= PENALTY_VALUE
        if (
            mass_flow[hotside_index2 - 1] * enthalpies[hotside_index2 - 1]
            < mass_flow[coldside_index2 - 1] * enthalpies[coldside_index2 - 1]
        ):
            print("Infeasible HX2.2")
            c= PENALTY_VALUE
        try:
            (
                Temperatures[hotside_index2],
                enthalpies[hotside_index2],
                entropies[hotside_index2],
                Temperatures[coldside_index2],
                enthalpies[coldside_index2],
                entropies[coldside_index2],
                q_hx[min(hotside_index2, coldside_index2)],
            ) = HX_calculation(
                Temperatures[hotside_index2 - 1],
                Pressures[hotside_index2 - 1],
                enthalpies[hotside_index2 - 1],
                Temperatures[coldside_index2 - 1],
                Pressures[coldside_index2 - 1],
                enthalpies[coldside_index2 - 1],
                approach_temp,
                hx_pdrop,
                mass_flow[hotside_index2],
                mass_flow[coldside_index2],
            )
        except:
            print("HX2 calculation error")
            c= PENALTY_VALUE
    # Temperatures[assumed_outlet] = Temperatures[real_inlet]+approach_temp
    # enthalpies[assumed_outlet],entropies[assumed_outlet] = enthalpy_entropy(Temperatures[assumed_outlet],Pressures[assumed_outlet])
    # while Temperatures.prod():
    #     for i in range(len(Temperatures)):
    #         if Temperatures[i] != 0:
    #             if i == len(Temperatures) - 1:
    #                 if Temperatures[0] == 0:
    #                     if equipment[0] == 1:
    #                         (
    #                             enthalpies[0],
    #                             entropies[0],
    #                             Temperatures[0],
    #                             w_tur[0],
    #                         ) = turbine(
    #                             Temperatures[i],
    #                             Pressures[i],
    #                             Pressures[0],
    #                             ntur,
    #                             mass_flow[0],
    #                         )
    #                     elif equipment[0] == 3:
    #                         (
    #                             enthalpies[0],
    #                             entropies[0],
    #                             Temperatures[0],
    #                             w_comp[0],
    #                         ) = compressor(
    #                             Temperatures[i],
    #                             Pressures[i],
    #                             Pressures[0],
    #                             ncomp,
    #                             mass_flow[0],
    #                         )
    #                     elif equipment[0] == 9:
    #                         enthalpies[0] = enthalpies[i]
    #                         entropies[0] = entropies[i]
    #                         Temperatures[0] = Temperatures[i]

    #             else:
    #                     if equipment[i + 1] == 1:
    #                         (
    #                             enthalpies[i + 1],
    #                             entropies[i + 1],
    #                             Temperatures[i + 1],
    #                             w_tur[i + 1],
    #                         ) = turbine(
    #                             Temperatures[i],
    #                             Pressures[i],
    #                             Pressures[i + 1],
    #                             ntur,
    #                             mass_flow[i + 1],
    #                         )
    #                     elif equipment[i + 1] == 3:
    #                         (
    #                             enthalpies[i + 1],
    #                             entropies[i + 1],
    #                             Temperatures[i + 1],
    #                             w_comp[i + 1],
    #                         ) = compressor(
    #                             Temperatures[i],
    #                             Pressures[i],
    #                             Pressures[i + 1],
    #                             ncomp,
    #                             mass_flow[i + 1],
    #                         )
        
    # if splitter == True:
    #     (Temperatures, enthalpies, entropies) = splitter_mixer_calc(
    #         Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
    #     )
    # if splitter2 == True:
    #     (Temperatures, enthalpies, entropies) = splitter_mixer_calc_2(
    #         Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
    #     )


    # if (
    #     hx_position.size != 0
    #     and Temperatures[hx_position[0] - 1] != 0
    #     and Temperatures[hx_position[1] - 1] != 0
    # ):
    #     hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
    #     if (
    #         Temperatures[hotside_index - 1]
    #         < Temperatures[coldside_index - 1] + approach_temp
    #     ):
    #         print("Infeasible HX1.1")
    #         c= PENALTY_VALUE
    #     # if (
    #     #     mass_flow[hotside_index - 1] * enthalpies[hotside_index - 1]
    #     #     < mass_flow[coldside_index - 1] * enthalpies[coldside_index - 1]
    #     # ):
    #     #     print("Infeasible HX1.2")
    #     #     c= PENALTY_VALUE
    #     try:
    #         (
    #             Temperatures[hotside_index],
    #             enthalpies[hotside_index],
    #             entropies[hotside_index],
    #             Temperatures[coldside_index],
    #             enthalpies[coldside_index],
    #             entropies[coldside_index],
    #             q_hx[min(hotside_index, coldside_index)],
    #         ) = HX_calculation(
    #             Temperatures[hotside_index - 1],
    #             Pressures[hotside_index - 1],
    #             enthalpies[hotside_index - 1],
    #             Temperatures[coldside_index - 1],
    #             Pressures[coldside_index - 1],
    #             enthalpies[coldside_index - 1],
    #             approach_temp,
    #             hx_pdrop,
    #             mass_flow[hotside_index],
    #             mass_flow[coldside_index],
    #         )
    #     except:
    #         print("HX1 calculation error")
    #         c= PENALTY_VALUE


    # if (
    #     hx2_position.size != 0
    #     and Temperatures[hx2_position[0] - 1] != 0
    #     and Temperatures[hx2_position[1] - 1] != 0
    # ):
    #     hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
    #     if (
    #         Temperatures[hotside_index2 - 1]
    #         < Temperatures[coldside_index2 - 1] + approach_temp_2
    #     ):
    #         print("Infeasible HX2.1")
    #         c= PENALTY_VALUE
    #     if (
    #         mass_flow[hotside_index2 - 1] * enthalpies[hotside_index2 - 1]
    #         < mass_flow[coldside_index2 - 1] * enthalpies[coldside_index2 - 1]
    #     ):
    #         print("Infeasible HX2.2")
    #         c= PENALTY_VALUE
    #     try:
    #         (
    #             Temperatures[hotside_index2],
    #             enthalpies[hotside_index2],
    #             entropies[hotside_index2],
    #             Temperatures[coldside_index2],
    #             enthalpies[coldside_index2],
    #             entropies[coldside_index2],
    #             q_hx[min(hotside_index2, coldside_index2)],
    #         ) = HX_calculation(
    #             Temperatures[hotside_index2 - 1],
    #             Pressures[hotside_index2 - 1],
    #             enthalpies[hotside_index2 - 1],
    #             Temperatures[coldside_index2 - 1],
    #             Pressures[coldside_index2 - 1],
    #             enthalpies[coldside_index2 - 1],
    #             approach_temp,
    #             hx_pdrop,
    #             mass_flow[hotside_index2],
    #             mass_flow[coldside_index2],
    #         )
    #     except:
    #         print("HX2 calculation error")
    #         c= PENALTY_VALUE
    # Temperatures[assumed_outlet],enthalpies[assumed_outlet],entropies[assumed_outlet] = 0,0,0
Temperatures,equipment

In [ ]:
real_inlet = np.where(Temperatures[hx2_position-1]!=0)[0].item()
assumed_inlet 


In [ ]:
for (x,y,z,m) in zip(equipment,Pressures,Temperatures,mass_flow):
    print(f"Equipment = {x} Mass flow = {m:.2f} Pressure = {y/1e5} Temperature = {z:.2f}")
print(lcoe)

In [ ]:
for index in cooler_position:
    if Temperatures[index] >= Temperatures[index - 1]:
        # print("Infeasible Cooler")
        c= PENALTY_VALUE
enthalpies, entropies, q_cooler = cooler_calculation(
    cooler_position,
    Temperatures,
    Pressures,
    enthalpies,
    entropies,
    q_cooler,
    cooler_pdrop,
    mass_flow,
)

for index in heater_position:
    if Temperatures[index] <= Temperatures[index - 1]:
        # print("Infeasible Temperatures for heater")
        c= PENALTY_VALUE
enthalpies, entropies, q_heater = heater_calculation(
    heater_position,
    Temperatures,
    Pressures,
    enthalpies,
    entropies,
    q_heater,
    heater_pdrop,
    mass_flow,
)

total_heat = sum(q_heater)
fg_tout = fg_calculation(fg_m, total_heat)
cost_tur = turbine_econ(w_tur, Temperatures, cost_tur)
cost_comp = comp_econ(w_comp, cost_comp)
cost_cooler = cooler_econ(q_cooler, Temperatures, cost_cooler)
if np.all(cost_cooler == 0):
    c= PENALTY_VALUE
try:
    cost_heater, fg_mlist, fg_tinlist, fg_toutlist, descending_temp = heater_econ(
        equipment,
        q_heater,
        Temperatures,
        cost_heater,
        heater_position,
        total_heat,
        fg_m,
        fg_tin,
    )
except:
    # print("Heater calculation error")
    c= PENALTY_VALUE
if hx_position != []:
    cost_hx = hx_econ(q_hx, Temperatures, cost_hx, hotside_index, coldside_index)
    if np.all(cost_hx == 0):
        c= PENALTY_VALUE
if hx2_position != []:
    cost_hx2 = hx_econ(q_hx, Temperatures, cost_hx, hotside_index2, coldside_index2)
    if np.all(cost_hx == 0):
        c= PENALTY_VALUE
pec = cost_tur + cost_hx + cost_cooler + cost_comp + cost_heater+cost_hx2
prod_capacity = (sum(w_tur) - sum(w_comp)) / 1e6
zk, cfueltot, lcoe = economics(pec, prod_capacity)
# Exergy Analysis
exergies, e_fgin, e_fgout = exergy_calculation(
    mass_flow,
    enthalpies,
    entropies,
    q_heater,
    fg_mlist,
    fg_tinlist,
    fg_toutlist,
    equipment_length,
)
if hx_position == []:
    hotside_index = 0
    coldside_index = 0
if hx2_position == []:
    hotside_index2 = 0
    coldside_index2 = 0
m1, m2 = exergoeconomic_calculation(
        equipment,
        Temperatures,
        enumerated_equipment,
        exergies,
        w_tur,
        w_comp,
        descending_temp,
        e_fgin,
        e_fgout,
        zk,
        hotside_index,
        coldside_index,
        hotside_index2,
        coldside_index2,
    )
try:
    costs = np.linalg.solve(m1, m2)
except:
    print("Matrix solution problem")
    c= PENALTY_VALUE
Closs = costs[equipment_length + 1] * min(x for x in e_fgout if x != 0)
Cfuel = costs[equipment_length] * FGINLETEXERGY
Ztot = sum(zk)
Cproduct = Cfuel + Ztot - Closs
Ep = sum(w_tur) - sum(w_comp)
for i, j in enumerated_equipment:
    if j == 2:
        dissipation[i] = costs[i] * (exergies[i - 1] - exergies[i]) + zk[i]
Cdiss = sum(dissipation)
lcoe_calculated = (costs[-1] * Ep + Cdiss + Closs) / (Ep / 1e6)
c = lcoe_calculated
thermal_efficiency = (Ep) / 40.53e6
if thermal_efficiency < 0.1575:
    j = 100 * (0.30 - thermal_efficiency)
else:
    j = c + 1 * max(0, 0.1 - sum(q_hx) / sum(q_heater))
print(
    f"""
Equipment = {equipment}
Turbine Pratio = {tur_pratio[np.where(tur_pratio>1.0001)[0]]}
Turbine Output = {w_tur[np.where(w_tur>0)[0]]/1e6}MW
Compressor Pratio = {comp_pratio[np.where(comp_pratio>1.0001)[0]]}
Compressor Input = {w_comp[np.where(w_comp>0)[0]]/1e6}MW
Split Ratio = {split_ratio:.2f} mass_flow = {mass_flow[:5]}
Temperatures = {Temperatures}   Tstack = {min(fg_toutlist[np.where(fg_toutlist>0)[0]]):.2f}    DT = {approach_temp}
Pressures =    {Pressures/1e5} bar
Equipment Cost = {pec/1e3}
Equipment Duty = Qheater={q_heater[np.where(q_heater>0)[0]]/1e6}MW   Qcooler={q_cooler[np.where(q_cooler>0)[0]]/1e6}MW   Qhx={q_hx[np.where(q_hx>0)[0]]/1e6}MW
Objective Function value = {c:.2f} LCOE = {lcoe:.2f} LCOEX = {lcoe_calculated:.2f}
Exergy of streams = {exergies/1e6}MW
Exergy of FG streams = {e_fgin[np.where(e_fgin>0)]/1e6}
                        {e_fgout[np.where(e_fgout>0)]/1e6}
                        {fg_mlist[np.where(e_fgout>0)]}
Exergy costing of streams = {costs/3600*1e9} $/GJ
Total PEC = {sum(pec):.2f} $
Total Zk  = {sum(zk):.2f} $/h
Cdiss = {Cdiss:.2f} Cl = {Closs:.2f} Cp ={costs[-1]*Ep:.2f} LCOE = {lcoe:.2f} LCOEX = {lcoe_calculated:.2f}
Cp/Ep = {Cproduct/(Ep/1e6):.2f}
Thermal efficiency = {thermal_efficiency*100:.2f}%
Heat recuperation ratio = {sum(q_hx)/(sum(q_heater))*100:.2f}
j = {j:.2f}
    """
)